In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# importing API to connect to NFL data sources like nflfastR, nfldata, dynastyprocess, and Draft Scout.
import nfl_data_py as nfl 

import requests
from bs4 import BeautifulSoup

## Data ingestion

In [46]:
# fantasy football datasets; results from each season

# ff22 = pd.read_csv('C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/2022_ff_stats.csv')
# can't find similar ff22 data,so going to scrape from pro football reference

ff21 = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2021.csv', index_col = 0)

ff20 = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2020.csv')

ff19 = pd.read_csv('https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2019.csv', index_col = 0)

# all nfl team names and abbreviations

nflteams = pd.read_csv('https://gist.githubusercontent.com/cnizzardini/13d0a072adb35a0d5817/raw/f315c97c7677845668a9c26e9093d0d550533b00/nfl_teams.csv',
                       index_col=0)
# team stats and rankings from each game per season

team19 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/2019_NFL_COMBINE.csv", index_col = 0)
team20 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/2020_NFL_COMBINE.csv", index_col = 0)
team21 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/2021_NFL_COMBINE.csv", index_col = 0)
team22 = pd.read_csv("C:/Users/steph/OneDrive/Documents/Data Projects/2022_NFL_COMBINE.csv", index_col = 0)

# Incoming rookie data

rosters19 = nfl.import_rosters([2019], ['team', 'player_name','position', 'player_id', 
                                        'height', 'weight', 'years_exp'])

rosters20 = nfl.import_rosters([2020], ['team', 'player_name', 'position', 'player_id', 
                                        'height', 'weight', 'years_exp'])

rosters21 = nfl.import_rosters([2021], ['team', 'player_name', 'position', 'player_id', 'height', 
                                        'weight', 'years_exp'])

rosters22 = nfl.import_rosters([2022], ['team', 'player_name', 'position', 'player_id', 'height', 
                                        'weight', 'years_exp'])


# seasonal data

season19 = nfl.import_seasonal_data([2019])
season20 = nfl.import_seasonal_data([2020])
season21 = nfl.import_seasonal_data([2021])
season22 = nfl.import_seasonal_data([2022])

# Injury Data?
injuries19 = nfl.import_injuries([2019])
injuries20 = nfl.import_injuries([2020])
injuries21 = nfl.import_injuries([2021])
injuries22 = nfl.import_injuries([2022])

# QBR data
# defaults are NFL and seasonal

qbr19 = nfl.import_qbr([2019])
qbr20 = nfl.import_qbr([2020])
qbr21 = nfl.import_qbr([2021])
qbr22 = nfl.import_qbr([2022])

# Next gen stat data

ngs_rushing = nfl.import_ngs_data('rushing', [2019, 2020, 2021, 2022])

ngs_receiving = nfl.import_ngs_data('receiving', [2019, 2020, 2021, 2022])

ngs_passing = nfl.import_ngs_data('passing', [2019, 2020, 2021, 2022])

# team wins

# this is actually just projected win total for gambling

team_wins = nfl.import_win_totals([2019, 2020, 2021, 2022])

# other data ideas:

# PFF Data (OL rankings, etc)
# contract data?


# clean data of team name, player name, and other descriptive data
# clean = nfl.clean_nfl_data(insert_Df)


C:\Users\steph\miniconda3\lib\site-packages\nfl_data_py\__init__.py:290: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  season_stats = all_stats.drop(['recent_team', 'week'], axis=1).groupby(
C:\Users\steph\miniconda3\lib\site-packages\nfl_data_py\__init__.py:309: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  szn = data.groupby(['player_id', 'season', 'season_type']).sum().reset_index().merge(
C:\Users\steph\miniconda3\lib\site-packages\nfl_data_py\__init__.py:290: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Ei

In [47]:
team22.head(25)

,Week,Day,Date,Unnamed: 5,Opp,Tm,Opp.1,1stD,TotYd,PassY,...,TO,1stD.1,TotYd.1,PassY.1,RushY.1,TO.1,Offense,Defense,Sp. Tms,Home_team
0,1,Sun,11-Sep,L,Chicago Bears,10.0,19.0,17.0,331.0,155.0,...,2.0,15.0,204.0,105.0,99.0,1.0,-11.26,-0.67,0.42,San Francisco 49ers
1,2,Sun,18-Sep,W,Seattle Seahawks,27.0,7.0,25.0,373.0,184.0,...,NaN,14.0,216.0,180.0,36.0,3.0,16.39,9.18,-7.67,San Francisco 49ers
2,3,Sun,25-Sep,L,Denver Broncos,10.0,11.0,13.0,267.0,179.0,...,3.0,14.0,261.0,160.0,101.0,NaN,-16.39,13.59,-1.26,San Francisco 49ers
3,4,Mon,3-Oct,W,Los Angeles Rams,24.0,9.0,13.0,327.0,239.0,...,NaN,21.0,257.0,200.0,57.0,2.0,10.05,11.56,-7.93,San Francisco 49ers
4,5,Sun,9-Oct,W,Carolina Panthers,37.0,15.0,22.0,397.0,244.0,...,1.0,15.0,308.0,244.0,64.0,1.0,16.04,13.22,-7.82,San Francisco 49ers
5,6,Sun,16-Oct,NaN,Atlanta Falcons,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco 49ers
6,7,Sun,23-Oct,NaN,Kansas City Chiefs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco 49ers
7,8,Sun,30-Oct,NaN,Los Angeles Rams,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco 49ers
8,9,NaN,NaN,NaN,Bye Week,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco 49ers
9,10,Sun,13-Nov,NaN,Los Angeles Chargers,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco 49ers


The team data is only for the first 6 weeks of the 2022 season. this is something to keep an eye out for / keep in mind. perhaps, we supplement with other data sources.

In [48]:
# let's check out injury data

year = [19, 20, 21, 22]
injuries_filtered = []

for i in range(len(year)):
    season = year[i]
    injuries = f'injuries{season}'
    inj = globals()[injuries]
    fantasy_pos = {'QB', 'RB', 'WR', 'TE'}
    
    inj_filtered = inj[inj['position'].isin(fantasy_pos)]
    injuries_filtered.append(inj_filtered)

injuries19 = injuries_filtered[0]
injuries20 = injuries_filtered[1]
injuries21 = injuries_filtered[2]
injuries22 = injuries_filtered[3]

injuries = pd.concat([injuries19, injuries20, injuries21, injuries22])

In [49]:
injuries.loc[injuries['first_name'] == 'Tyreek']

,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
349,2019.0,REG,KC,2.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Did Not Participate In Practice,2019-09-13 12:48:28+00:00
651,2019.0,REG,KC,3.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Did Not Participate In Practice,2019-09-20 12:03:15+00:00
1002,2019.0,REG,KC,4.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Did Not Participate In Practice,2019-09-27 11:58:29+00:00
1310,2019.0,REG,KC,5.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Out,Shoulder,None,Limited Participation in Practice,2019-10-04 12:38:41+00:00
1590,2019.0,REG,KC,6.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,Shoulder,None,Questionable,Shoulder,None,Limited Participation in Practice,2019-10-11 12:41:27+00:00
1889,2019.0,REG,KC,7.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Shoulder,None,Full Participation in Practice,2019-10-16 10:41:11+00:00
3699,2019.0,REG,KC,13.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Hamstring,None,Full Participation in Practice,2019-11-29 12:27:34+00:00
4040,2019.0,REG,KC,14.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Hamstring,None,Full Participation in Practice,2019-12-04 14:04:32+00:00
4709,2019.0,REG,KC,16.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Shoulder,None,Full Participation in Practice,2019-12-20 12:20:44+00:00
945,2020.0,REG,KC,4.0,00-0033040,WR,Tyreek Hill,Tyreek,Hill,None,None,None,Heel,None,Full Participation in Practice,2020-10-02 12:15:01+00:00


It seems these are the once a week injury reports. One way to aggregate this, perhaps, is to count the amount of 'Out's,'Questionable's, and both, grouped by season and player.

In [50]:
injuries['out_occured'] = injuries['report_status'].apply(lambda x: 1 if x == 'Out' else 0)
injuries['questionable_occured'] = injuries['report_status'].apply(lambda x: 1 if x == 'Questionable' else 0)

injuries['games_out'] = injuries.groupby(['gsis_id', 'season'])['out_occured'].transform('sum')
injuries['games_questionable'] = injuries.groupby(['gsis_id', 'season'])['questionable_occured'].transform('sum')
injuries['games_out_or_questionable'] = injuries['games_out'] + injuries['games_questionable']


injuries_by_season = injuries.groupby(['gsis_id', 'season'])\
                  [['full_name', 'games_out', 'games_questionable', 'games_out_or_questionable']]\
                 .agg({'full_name': 'first', 'games_out': 'mean',
                       'games_questionable': 'mean', 'games_out_or_questionable': 'mean'})\
                    .reset_index()

In [51]:
injuries_by_season

,gsis_id,season,full_name,games_out,games_questionable,games_out_or_questionable
0,00-0019596,2019.0,Tom Brady,0.0,1.0,1.0
1,00-0019596,2020.0,Tom Brady,0.0,0.0,0.0
2,00-0019596,2021.0,Tom Brady,0.0,0.0,0.0
3,00-0019596,2022.0,Tom Brady,0.0,0.0,0.0
4,00-0020531,2019.0,Drew Brees,5.0,1.0,6.0
...,...,...,...,...,...,...
1693,00-0038117,2022.0,Wan'Dale Robinson,4.0,2.0,6.0
1694,00-0038120,2022.0,Breece Hall,0.0,0.0,0.0
1695,00-0038124,2022.0,Christian Watson,2.0,4.0,6.0
1696,00-0038129,2022.0,Cade Otton,0.0,0.0,0.0


### Getting Fantasy data for 2022 (ff22)

In [52]:
### utilizing beautifulsoup to scrape ff22 data!

import requests
from bs4 import BeautifulSoup

# URL of the fantasy football page
url = 'https://www.pro-football-reference.com/years/2022/fantasy.htm'

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object with the content of the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the fantasy football data
table = soup.find('table', {'id': 'fantasy'})

# Extract the column names from the table headers
subheader_row = table.find('thead').find_all('tr')[1]
column_names = [subheader.text for subheader in subheader_row.find_all('th')][:-1]

# Find all rows in the table (excluding the header row)
rows = table.find_all('tr')[1:]

# Define a list to store the data
data = []

# Iterate over each row and extract the data
for row in rows:
    # Find all cells in the row
    cells = row.find_all('td')

    # Check if the row has the expected number of cells
    if len(cells) >= len(column_names):
        # Extract the data from the cells
        row_data = [cell.text for cell in cells[:len(column_names)]]

        # Append the data to the list
        data.append(row_data)

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns = column_names)

# Save the DataFrame to a CSV file
df.to_csv('fantasy_data.csv', index=False)

In [53]:
# it seems the columns were off by 1
# shifting with code below, while dropping rank column

ff22 = df.shift(axis = 1).drop(columns = 'Rk')

In [54]:
ff22.head()

,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,TD,...,FL,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank
0,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,41,...,0,4,1,2,416,417.4,435.4,428.9,136,1
1,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,0,...,1,12,,,275,328.3,335.3,301.8,127,1
2,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,1,...,0,13,,,271,356.4,362.4,313.9,123,2
3,Derrick Henry*,TEN,RB,28,16,16,2,2,4,1,...,3,13,,,270,302.8,311.8,286.3,122,3
4,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,0,...,0,9,1,,241,368.7,371.7,304.7,119,1


In [55]:
# looking at ff21 (similar columns as ff20 and ff19)
pd.set_option('display.max_columns', None)

ff21.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


In [56]:
# looking at ff22 -- these have different columns than ff19 - ff21

ff22.head()

,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,TD,Int,Att,Yds,Y/A,TD,Tgt,Rec,Yds,Y/R,TD,Fmb,FL,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank
0,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,41,12,61,358,5.87,4,1,1,6,6.00,0,5,0,4,1,2,416,417.4,435.4,428.9,136,1
1,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,0,0,340,1653,4.86,12,64,53,400,7.55,0,3,1,12,,,275,328.3,335.3,301.8,127,1
2,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,1,0,244,1139,4.67,8,108,85,741,8.72,5,1,0,13,,,271,356.4,362.4,313.9,123,2
3,Derrick Henry*,TEN,RB,28,16,16,2,2,4,1,0,349,1538,4.41,13,41,33,398,12.06,0,6,3,13,,,270,302.8,311.8,286.3,122,3
4,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,0,0,4,24,6.00,1,184,128,1809,14.13,8,0,0,9,1,,241,368.7,371.7,304.7,119,1


In [57]:
# due to main header not being included with, need to differentiate between identical column names like Yds or TD

# writing a function for duplicate names
def _maybe_dedup_names(self, names):
    if self.mangle_dupe_cols:
        names = list(names)  # to index
        counts = {}
        
        # getting index and col value
        for i, col in enumerate(names):
            cur_count = counts.get(col, 0)

            if cur_count > 0:
                names[i] = '%s.%d' % (col, cur_count)

            counts[col] = cur_count + 1

    return names

ff22.columns =pd.io.parsers.base_parser.ParserBase({'names':ff22.columns, 'usecols':None})._maybe_dedup_names(ff22.columns)

In [58]:
# since ff22 is from a dif source than ff19-ff21, we'll rename columns
# TD.3 is total touchdowns, incld special teams

ff22.rename(columns = {"FantPos":"Pos", "Att" : "PassingAtt", "Yds": "PassingYds",
            "TD": "PassingTD", "Att.1": "RushingAtt", "Yds.1": "RushingYds", "TD.1": "RushingTD", 
            "TD.1": "RushingTD", "Yds.2": "ReceivingYds", "TD.2": "ReceivingTD", 
                       "FantPt": "FantasyPoints", "TD.3": "TotalTds_22", "Fmb": "Fumbles",
                      "FL": "FumblesLost"}, inplace = True)

# ff22.drop('PTS/G', axis =1 ,inplace =True)

ff22.head()

,Player,Tm,Pos,Age,G,GS,Cmp,PassingAtt,PassingYds,PassingTD,Int,RushingAtt,RushingYds,Y/A,RushingTD,Tgt,Rec,ReceivingYds,Y/R,ReceivingTD,Fumbles,FumblesLost,TotalTds_22,2PM,2PP,FantasyPoints,PPR,DKPt,FDPt,VBD,PosRank
0,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,41,12,61,358,5.87,4,1,1,6,6.00,0,5,0,4,1,2,416,417.4,435.4,428.9,136,1
1,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,0,0,340,1653,4.86,12,64,53,400,7.55,0,3,1,12,,,275,328.3,335.3,301.8,127,1
2,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,1,0,244,1139,4.67,8,108,85,741,8.72,5,1,0,13,,,271,356.4,362.4,313.9,123,2
3,Derrick Henry*,TEN,RB,28,16,16,2,2,4,1,0,349,1538,4.41,13,41,33,398,12.06,0,6,3,13,,,270,302.8,311.8,286.3,122,3
4,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,0,0,4,24,6.00,1,184,128,1809,14.13,8,0,0,9,1,,241,368.7,371.7,304.7,119,1


### Getting data on Fantasy Rankings

In [59]:
# 2023 rankings (current year)

# URL of the website
rank_url = "https://fantasyfootballcalculator.com/adp/half-ppr/12-team/all"

# Send a GET request to the URL
response = requests.get(rank_url)

# Create a BeautifulSoup object with the content of the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the main table on the page
table = soup.find("table", class_="table adp")

# Find all rows in the table (excluding the header row)
rows = table.find_all("tr")[1:]

# Define lists to store the data
data = []
header = []

# Extract the column names from the header row
for th in table.find_all("th"):
    header.append(th.text.strip())

# Iterate over each row and extract the data
for row in rows:
    cells = row.find_all("td")
    row_data = [cell.text.strip() for cell in cells]
    data.append(row_data)

# Create a DataFrame from the data
ranks_23 = pd.DataFrame(data, columns=header)

# Print the DataFrame
print(ranks_23)

       #   Pick                 Name  Pos Team Bye Overall Std.Dev   High  \
0      1   1.02  Christian McCaffrey   RB   SF   9     1.6     0.8   1.01   
1      2   1.02     Justin Jefferson   WR  MIN  13     2.2     0.9   1.01   
2      3   1.02        Austin Ekeler   RB  LAC   5     2.2     0.8   1.01   
3      4   1.04        Ja'Marr Chase   WR  CIN   7     4.0     1.2   1.01   
4      5   1.05       Saquon Barkley   RB  NYG  13     5.2     1.3   1.02   
..   ...    ...                  ...  ...  ...  ..     ...     ...    ...   
217  218  15.11  Kansas City Defense  DEF   KC  10   179.4    18.0  12.07   
218  219  15.12         Robert Woods   WR  HOU   7   180.3    22.1  12.10   
219  220  16.02        Greg Zuerlein   PK  NYJ   7   182.4    31.9  10.04   
220  221  16.05    Cleveland Defense  DEF  CLE   5   184.9    23.5  12.12   
221  222  16.07  LA Chargers Defense  DEF  LAC   5   186.5    12.5  14.10   

       Low TimesDrafted Graph  
0     1.05          375        
1     1.04 

In [60]:
# let's keep only relevant positions (QB, RB, WR, and TE)

ranks_23 = ranks_23.loc[ranks_23['Pos'].isin(['QB', 'RB', 'TE', 'WR'])]

# let's keep only relevant columns
ranks_23 = ranks_23[['#', 'Name', 'Pos', 'Team']]
ranks_23.rename(columns = {'#': 'adp'}, inplace = True)

In [61]:
# let's do it for the rest of the seasons - 2019 thru 2022

# List of years
years = [2019, 2020, 2021, 2022]

# Define lists to store the data
all_data = []

# Iterate over each year
for year in years:
    # Construct the URL for the specific year
    url = f"https://fantasyfootballcalculator.com/adp/half-ppr/12-team/all/{year}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object with the content of the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the main table on the page
    table = soup.find("table", class_="table adp")

    # Find all rows in the table (excluding the header row)
    rows = table.find_all("tr")[1:]

    # Extract the column names from the header row
    header = [th.text.strip() for th in table.find_all("th")]

    # Extract the data from each row
    data = []
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        row_data.append(year) # add year as a column
        data.append(row_data)
        

    # Append the data to the overall list
    all_data.extend(data)

# adding an extra column
header.append("season")

# Create a DataFrame from the collected data
ranks_19_thru_22 = pd.DataFrame(all_data, columns= header)

# Print the DataFrame
print(ranks_19_thru_22)

       #   Pick                  Name  Pos Team Overall Std.Dev   High    Low  \
0      1   1.01        Saquon Barkley   RB  NYG     1.3     0.6   1.01   1.05   
1      2   1.03          Alvin Kamara   RB   NO     2.7     0.8   1.01   1.06   
2      3   1.03   Christian McCaffrey   RB  CAR     3.1     1.1   1.01   1.07   
3      4   1.04       Ezekiel Elliott   RB  DAL     3.8     1.1   1.01   1.06   
4      5   1.06         David Johnson   RB  ARI     6.2     1.6   1.02   1.10   
..   ...    ...                   ...  ...  ...     ...     ...    ...    ...   
745  120  12.12   LA Chargers Defense  DEF  LAC   144.1    14.9  10.01  14.09   
746  121  13.03       Harrison Butker   PK   KC   146.9    16.7   9.12  16.07   
747  122  13.07  Indianapolis Defense  DEF  IND   150.7    18.8  10.01  17.01   
748  123  13.07        Raheem Mostert   RB  MIA   151.1    17.1  10.03  15.12   
749  124  14.02           Dawson Knox   TE  BUF   157.9    26.9   8.01  17.12   

    TimesDrafted     season

In [62]:
# let's keep only relevant positions (QB, RB, WR, and TE)

ranks_19_thru_22_rel = ranks_19_thru_22.loc[ranks_19_thru_22['Pos'].isin(['QB', 'RB', 'TE', 'WR'])]

# let's keep only relevant columns
ranks_19_thru_22_rel = ranks_19_thru_22_rel[['#', 'Name', 'Pos', 'Team', 'season']]
ranks_19_thru_22_rel.rename(columns = {'#': 'adp'}, inplace = True)

# let's split by season
ranks_19 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2019]
ranks_20 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2020]
ranks_21 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2021]
ranks_22 = ranks_19_thru_22_rel.loc[ranks_19_thru_22_rel['season'] == 2022]


In [63]:
ranks_19_thru_22.tail()

,#,Pick,Name,Pos,Team,Overall,Std.Dev,High,Low,TimesDrafted,,season
745,120,12.12,LA Chargers Defense,DEF,LAC,144.1,14.9,10.01,14.09,53,,2022
746,121,13.03,Harrison Butker,PK,KC,146.9,16.7,9.12,16.07,55,,2022
747,122,13.07,Indianapolis Defense,DEF,IND,150.7,18.8,10.01,17.01,52,,2022
748,123,13.07,Raheem Mostert,RB,MIA,151.1,17.1,10.03,15.12,57,,2022
749,124,14.02,Dawson Knox,TE,BUF,157.9,26.9,8.01,17.12,61,,2022


2022 data is limited here as well, only the first 124 adp players are here.

In [64]:
# QBR peek

qbr19.sample(5)

,season,season_type,game_week,team_abb,player_id,name_short,rank,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,penalty,qbr_raw,sack,name_first,name_last,name_display,headshot_href,team,qualified
1017,2019,Playoffs,Season Total,HOU,16016,K. Stills,NaN,0.0,-0.8,1,-0.5,-1.0,0.0,0,0.0,0.0,-0.5,Kenny,Stills,Kenny Stills,https://a.espncdn.com/i/headshots/nfl/players/...,Texans,False
1003,2019,Playoffs,Season Total,SF,16760,J. Garoppolo,8.0,45.8,-1.3,73,4.9,2.8,-0.3,0,0.1,43.8,-2.3,Jimmy,Garoppolo,Jimmy Garoppolo,https://a.espncdn.com/i/headshots/nfl/players/...,49ers,True
973,2019,Regular,Season Total,ARI,2508176,D. Johnson,NaN,100.0,0.7,1,0.7,0.0,0.7,0,0.0,100.0,0.0,David,Johnson,David Johnson,https://a.espncdn.com/i/headshots/nfl/players/...,Cardinals,False
1010,2019,Playoffs,Season Total,BUF,16804,J. Brown,NaN,100.0,0.6,1,0.7,0.7,0.0,0,0.0,100.0,0.0,John,Brown,John Brown,https://a.espncdn.com/i/headshots/nfl/players/...,Bills,False
948,2019,Regular,Season Total,SF,16760,J. Garoppolo,12.0,60.8,20.8,593,69.3,48.5,-0.2,0,5.0,62.1,-15.9,Jimmy,Garoppolo,Jimmy Garoppolo,https://a.espncdn.com/i/headshots/nfl/players/...,49ers,True


## Data Cleaning

Need to clean player name from *, + and/or both.

Merge in QBR data to QB data; ngs_rushing to qb and rb

need a dataset of predictor variables for final -- previous year's players (2021) plus rookies (2022) draft.

Need to bring in a dataset of rookies 

Need a dataset of team ranks from previous year (2021)

Need to group by position, as points scored and value is dependent on position.


We'll use 19 and 20 as training years; 21 as test year. 

Final model will use previous 2 season data, and other, to predict upcoming season data


### Fantasy Points DataSources

In [65]:
ff20.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Derrick Henry,TEN,RB,26,16,16,31,19,0,0,0,2027,17,378,114,0,314.0,0,3,2
1,Alvin Kamara,NOR,RB,25,15,10,107,83,0,0,0,932,16,187,756,5,295.0,0,1,0
2,Dalvin Cook,MIN,RB,25,14,14,54,44,0,0,0,1557,16,312,361,1,294.0,0,5,3
3,Travis Kelce,KAN,TE,31,15,15,145,105,4,0,2,0,0,0,1416,11,208.0,0,1,1
4,Davante Adams,GNB,WR,28,14,14,149,115,0,0,0,0,0,0,1374,18,243.0,0,1,1


In [66]:
ff21.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Jonathan Taylor*+,IND,RB,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,Cooper Kupp*+,LAR,WR,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,Deebo Samuel*+,SFO,WR,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


In [67]:
ff19.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
0,Christian McCaffrey,CAR,RB,23.0,16.0,16.0,142.0,116.0,0.0,0.0,2.0,1387.0,15.0,287.0,1005.0,4.0,469.20,0.0,1.0,0.0
1,Lamar Jackson,BAL,QB,22.0,15.0,15.0,0.0,0.0,3127.0,36.0,401.0,1206.0,7.0,176.0,0.0,0.0,415.68,6.0,9.0,2.0
2,Derrick Henry,TEN,RB,25.0,15.0,15.0,24.0,18.0,0.0,0.0,0.0,1540.0,16.0,303.0,206.0,2.0,294.60,0.0,5.0,3.0
3,Aaron Jones,GNB,RB,25.0,16.0,16.0,68.0,49.0,0.0,0.0,0.0,1084.0,16.0,236.0,474.0,3.0,314.80,0.0,3.0,2.0
4,Ezekiel Elliott,DAL,RB,24.0,16.0,16.0,71.0,54.0,0.0,0.0,0.0,1357.0,12.0,301.0,420.0,2.0,311.70,0.0,3.0,2.0


In [68]:
# some players have '+' or '*' in their name in some dfs to indicate if they're all-pro or pro-bowlers
# we want to get rid of this so all names are just their names

import re

ff_dfs = [ff22, ff21, ff20, ff19]

text_to_remove = ['+', '*']

# to ensure we escape special characters so there are no errors
text_to_remove_escaped = [re.escape(text) for text in text_to_remove]

for df in ff_dfs:
    df['Player'] = df['Player'].str.replace('|'.join(text_to_remove_escaped), '',regex= True )
    
ff21.head()

,Player,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost
0,Jonathan Taylor,IND,RB,22,17,17,51.0,40.0,0.0,0.0,0.0,1811.0,18.0,332.0,360.0,2.0,333.0,0.0,4.0,2.0
1,Cooper Kupp,LAR,WR,28,17,17,191.0,145.0,0.0,0.0,1.0,18.0,0.0,4.0,1947.0,16.0,295.0,0.0,0.0,0.0
2,Deebo Samuel,SFO,WR,25,16,15,121.0,77.0,24.0,1.0,2.0,365.0,8.0,59.0,1405.0,6.0,262.0,0.0,4.0,2.0
3,Josh Allen,BUF,QB,25,17,17,0.0,0.0,4407.0,36.0,646.0,763.0,6.0,122.0,0.0,0.0,403.0,15.0,8.0,3.0
4,Austin Ekeler,LAC,RB,26,16,16,94.0,70.0,0.0,0.0,0.0,911.0,12.0,206.0,647.0,8.0,274.0,0.0,4.0,3.0


## Team data

### Create Team Stats DFs for each season

In [69]:
# ensure all teams are represented

team21['Home_team'].value_counts()

Los Angeles Rams        23
Cincinnati Bengals      23
San Francisco 49ers     22
Kansas City Chiefs      22
Tampa Bay Buccaneers    21
Buffalo Bills           21
Arizona Cardinals       20
Pittsburgh Steelers     20
Green Bay Packers       20
Dallas Cowboys          20
Philadelphia Eagles     20
Tennessee Titans        20
Oakland Raiders         20
New England Patriots    20
Houston Texans          18
Seattle Seahawks        18
New Orleans Saints      18
Baltimore Ravens        18
Carolina Panthers       18
New York Giants         18
Detroit Lions           18
New York Jets           18
Jacksonville Jaguars    18
Chicago Bears           18
Atlanta Falcons         18
Miami Dolphins          18
Indianapolis Colts      18
Los Angeles Chargers    18
Cleveland Browns        18
Denver Broncos          18
Minnesota Vikings       18
Name: Home_team, dtype: int64

The Washington Football Team (Commanders) are not represented in any of these team datasets.

In [70]:
# getting rid of bye week rows

teams = [team19, team20, team21, team22]

for i in range(len(teams)):
    teams[i] = teams[i][teams[i].Opp != 'Bye Week']

In [71]:
team19 = teams[0]
team20 = teams[1]
team21 = teams[2]
team22 = teams[3]

In [72]:
# weeks are objects in team19 - team21, changing team22 to that as well

team22['Week'] = team22['Week'].astype(str)

Important to note, 2021 is first year with 18 weeks, rather than 17.

In [73]:
# must get rid of division, wild card, conf champ, and superbowl

reg1920 = np.arange(1, 18).astype(str)
reg21 = np.arange(1,19).astype(str)

team19_reg = team19.loc[team19['Week'].isin(reg1920)]
team20_reg = team20.loc[team20['Week'].isin(reg1920)]
team21_reg = team21.loc[team21['Week'].isin(reg21)]
team22_reg = team22.loc[team22['Week'].isin(reg21)]

team19_reg['Week'].value_counts()

1     31
2     31
3     31
17    31
16    31
15    31
14    31
13    31
4     29
8     29
5     29
11    27
12    27
9     27
7     27
6     27
10    26
Name: Week, dtype: int64

In [74]:
# taking just the existing 6 weeks of this season

team22_reg = team22_reg.dropna(subset = ['Tm', '1stD'], how = 'all')

In [75]:
# keeping only relevant columns

relevant_team = ['Home_team',  'Unnamed: 5', 'Tm', '1stD', 'TotYd', 'PassY', 'RushY', 'TO',
           'Offense', 'Defense', 'Opp.1', 'TotYd.1', 'PassY.1', 'RushY.1', 'TO.1', 'Sp. Tms']

team19_reg = team19_reg[relevant_team]
team20_reg = team20_reg[relevant_team]
team21_reg = team21_reg[relevant_team]
team22_reg = team22_reg[relevant_team]

# adding in columns to show season
team19_reg['season'] = '2019'
team20_reg['season'] = '2020'
team21_reg['season'] = '2021'
team22_reg['season'] = '2022'

In [76]:
# concatenating all years together

team_all =pd.concat([team19_reg, team20_reg, team21_reg, team22_reg])

team_all.head(20)

,Home_team,Unnamed: 5,Tm,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Opp.1,TotYd.1,PassY.1,RushY.1,TO.1,Sp. Tms,season
0,San Francisco 49ers,W,31.0,17.0,256.0,158.0,98.0,2.0,-7.51,19.12,17.0,295.0,174.0,121.0,4.0,1.10,2019
1,San Francisco 49ers,W,41.0,27.0,571.0,312.0,259.0,1.0,27.34,1.14,17.0,316.0,291.0,25.0,1.0,-6.64,2019
2,San Francisco 49ers,W,24.0,26.0,436.0,268.0,168.0,5.0,-3.72,10.20,20.0,239.0,160.0,79.0,2.0,0.12,2019
4,San Francisco 49ers,W,31.0,22.0,446.0,171.0,275.0,NaN,12.38,22.56,3.0,180.0,78.0,102.0,4.0,-10.50,2019
5,San Francisco 49ers,W,20.0,22.0,331.0,232.0,99.0,2.0,-3.54,25.23,7.0,157.0,48.0,109.0,1.0,-5.01,2019
6,San Francisco 49ers,W,9.0,17.0,283.0,146.0,137.0,1.0,-1.99,14.12,0.0,154.0,50.0,104.0,1.0,-0.61,2019
7,San Francisco 49ers,W,51.0,24.0,388.0,156.0,232.0,1.0,15.84,30.05,13.0,230.0,100.0,130.0,3.0,-7.14,2019
8,San Francisco 49ers,W,28.0,21.0,411.0,310.0,101.0,NaN,18.45,-13.71,25.0,357.0,204.0,153.0,NaN,0.88,2019
9,San Francisco 49ers,L,24.0,21.0,302.0,215.0,87.0,3.0,-17.73,17.69,27.0,336.0,189.0,147.0,4.0,-4.23,2019
10,San Francisco 49ers,W,36.0,26.0,442.0,408.0,34.0,2.0,8.92,-0.09,26.0,266.0,131.0,135.0,2.0,1.17,2019


In [77]:
# getting 'dummies' for the amount of wins, ties and losses each team has per season

team_all = pd.get_dummies(team_all, columns = ['Unnamed: 5']).groupby(['Home_team', 'season'], as_index = False).sum()

# from here, creating a new column that sums up wins and ties

team_all['team_wins'] = team_all['Unnamed: 5_W'] + 0.5 * team_all['Unnamed: 5_T']
team_all.head()

,Home_team,season,Tm,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Opp.1,TotYd.1,PassY.1,RushY.1,TO.1,Sp. Tms,Unnamed: 5_L,Unnamed: 5_T,Unnamed: 5_W,team_wins
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10,1,5,5.5
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8,0,8,8.0
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6,0,11,11.0
3,Arizona Cardinals,2022,105.0,112.0,1761.0,1189.0,572.0,3.0,21.28,-34.67,123.0,1728.0,1241.0,487.0,6.0,-4.15,3,0,2,2.0
4,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9,0,7,7.0


In [78]:
team_all = team_all.groupby(['Home_team', 'season']).mean().reset_index()

team_all.head()

,Home_team,season,Tm,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Opp.1,TotYd.1,PassY.1,RushY.1,TO.1,Sp. Tms,Unnamed: 5_L,Unnamed: 5_T,Unnamed: 5_W,team_wins
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8.0,0.0,8.0,8.0
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6.0,0.0,11.0,11.0
3,Arizona Cardinals,2022,105.0,112.0,1761.0,1189.0,572.0,3.0,21.28,-34.67,123.0,1728.0,1241.0,487.0,6.0,-4.15,3.0,0.0,2.0,2.0
4,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9.0,0.0,7.0,7.0


In [79]:
# rename columns to ensure we're clear what are defensive and offensive stats

team_all.rename(columns = {'Home_team': 'Team', 'Unnamed: 5_L': 'losses',  'Unnamed: 5_T': 'ties',\
                           'Unnamed: 5_W': 'wins', 'Tm': 'Pts_scored',
                          'Opp.1':'Pts_allowed', 'TotYd.1': 'Yds_alowed',
                          'PassY.1': 'PassYds_allowed', 'RushY.1': 'Rushyds_allowed',
                          "TO.1": "Team_takeaways"}, inplace = True)

In [80]:
team_all.head()

,Team,season,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8.0,0.0,8.0,8.0
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6.0,0.0,11.0,11.0
3,Arizona Cardinals,2022,105.0,112.0,1761.0,1189.0,572.0,3.0,21.28,-34.67,123.0,1728.0,1241.0,487.0,6.0,-4.15,3.0,0.0,2.0,2.0
4,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9.0,0.0,7.0,7.0


In [81]:
nflteams.head()

,Name,Abbreviation,Conference,Division
ID,,,,
1,Arizona Cardinals,ARI,NFC,West
2,Atlanta Falcons,ATL,NFC,South
3,Baltimore Ravens,BAL,AFC,North
4,Buffalo Bills,BUF,AFC,East
5,Carolina Panthers,CAR,NFC,South


In [82]:
team_all_collective = team_all.merge(nflteams[['Name','Abbreviation']], left_on = 'Team', right_on = 'Name').drop(columns = 'Name')

team_all_collective.head()

,Team,season,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Abbreviation
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8.0,0.0,8.0,8.0,ARI
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6.0,0.0,11.0,11.0,ARI
3,Arizona Cardinals,2022,105.0,112.0,1761.0,1189.0,572.0,3.0,21.28,-34.67,123.0,1728.0,1241.0,487.0,6.0,-4.15,3.0,0.0,2.0,2.0,ARI
4,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9.0,0.0,7.0,7.0,ATL


## Roster data

Now that fantasy points for 3 seasons are relatively cleaned; before merging, let's try to clean up roster data

In [83]:
rosters19.sample(n=8)

,team,player_name,position,player_id,height,weight,years_exp
2428,PHI,Craig James,DB,00-0034310,70.0,195.0,1.0
589,CIN,Tony McRae,DB,00-0032934,70.0,185.0,3.0
903,DEN,Don Barclay,OL,00-0029187,NaN,305.0,NaN
1722,LAC,Justice Liggins,WR,00-0034038,NaN,209.0,NaN
52,ARI,J.R. Sweezy,OL,00-0029146,77.0,310.0,7.0
1883,MIA,Tre Watson,LB,00-0035349,NaN,241.0,NaN
768,DAL,D.J. White,DB,00-0032788,71.0,193.0,3.0
1380,JAX,Chuck Harris,LB,00-0035071,76.0,265.0,NaN


In [84]:
# We must get rid of non-offensive, fantasy irrelevant positions

# for this analysis, only focused on qb, rb, wr, te

ff_positions = ['QB', 'RB', 'WR', 'TE']

rosters19_rel = rosters19.loc[rosters19['position'].isin(ff_positions)]
rosters20_rel = rosters20.loc[rosters20['position'].isin(ff_positions)]
rosters21_rel = rosters21.loc[rosters21['position'].isin(ff_positions)]
rosters22_rel = rosters22.loc[rosters22['position'].isin(ff_positions)]

In [85]:
rosters19_rel.columns

Index(['team', 'player_name', 'position', 'player_id', 'height', 'weight',
       'years_exp'],
      dtype='object')

### Merging roster, fantasy, and adp data

Much of initial analysis and project has been without using the season data, rather fantasy data and bringing in next gen stats, etc. However, the season_data is possibly a cleaner way of going about it. 

As of 7/18/23, i will continue to utilze the fantasy data as originally planned, but do have this below code in in case I want to pivot or iterate later on.

In [86]:
# merging roster data on season data (due to it having player_id's available as a key)
# and bringing in rank data

ff19_rs = rosters19_rel.merge(season19, how = 'outer', on = 'player_id').merge(ranks_19[['adp', 'Name']], how = 'left', \
                    left_on = 'player_name', right_on = 'Name').merge(ff19[['GS', 'Player']], how = 'left', 
                                                                      left_on = 'player_name', right_on = 'Player')

ff20_rs = rosters20_rel.merge(season20, how = 'outer', on = 'player_id').merge(ranks_20[['adp', 'Name']], how = 'left', \
                    left_on = 'player_name', right_on = 'Name').merge(ff20[['GS', 'Player']], how = 'left', 
                                                                      left_on = 'player_name', right_on = 'Player')

ff21_rs = rosters21_rel.merge(season21, how = 'outer', on = 'player_id').merge(ranks_21[['adp', 'Name']], how = 'left', \
                    left_on = 'player_name', right_on = 'Name').merge(ff21[['GS', 'Player']], how = 'left', 
                                                                      left_on = 'player_name', right_on = 'Player')

ff22_rs = rosters22_rel.merge(season22, how = 'outer', on = 'player_id').merge(ranks_22[['adp', 'Name']], how = 'left', \
                    left_on = 'player_name', right_on = 'Name').merge(ff22[['GS', 'Player']], how = 'left', 
                                                                      left_on = 'player_name', right_on = 'Player')

# checking shape of each to ensure nothing is amiss

print(ff19_rs.shape, ff20_rs.shape, ff21_rs.shape, ff22_rs.shape)

(1035, 68) (883, 68) (886, 68) (846, 68)


In [87]:
ff19_rs.isna().sum()

team            46
player_name     46
position        46
player_id        1
height         193
              ... 
ppr_sh         420
adp            871
Name           871
GS             432
Player         432
Length: 68, dtype: int64

In [88]:
ff22_rs.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,adp,Name,GS,Player
666,PIT,Rod Williams,TE,00-0037451,76.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,ATL,Feleipe Franks,TE,00-0036825,78.0,245.0,1.0,2022.0,REG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.838100,0.0,0.0,2.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,-1.094847,0.0,0.000000,0.083333,0.262911,0.309038,0.0,0.0,0.0,2.0,0.039216,0.123620,0.000000,0.157720,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,1,Feleipe Franks
128,CAR,Derek Wright,WR,00-0037368,73.0,186.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,IND,Aaron Shampklin,RB,00-0037609,69.0,194.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,MIA,Raheem Mostert,RB,00-0031687,70.0,197.0,7.0,2022.0,REG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,181.0,891.0,3.0,1.0,1.0,44.0,-0.987838,0.0,31.0,42.0,202.0,2.0,0.0,0.0,48.0,179.0,11.0,1.435230,0.0,22.069048,1.283640,0.153992,2.033254,0.0,137.3,168.3,16.0,0.077348,0.009901,0.104373,0.123943,0.045291,0.068966,0.054455,0.056277,0.057128,0.050026,0.372007,0.115797,123,Raheem Mostert,14,Raheem Mostert


In [89]:
ff19_r = rosters19_rel.merge(ff19, how = 'outer', left_on = 'player_name', \
                             right_on = 'Player', suffixes = ('_19r', '_19'))\
                    .reset_index(drop = True).merge(ranks_19[['adp', 'Name']], \
                                            how = 'left', left_on = 'player_name',right_on = 'Name')

ff19_r.drop(columns = 'Player', inplace = True)

ff19_r.head()

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,adp,Name
0,ARI,Brett Hundley,QB,00-0031589,75.0,226.0,4.0,ARI,QB,26.0,3.0,0.0,0.0,0.0,49.0,0.0,11.0,41.0,0.0,7.0,0.0,0.0,6.06,0.0,0.0,0.0,NaN,NaN
1,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,0.0,ARI,QB,22.0,16.0,16.0,0.0,0.0,3722.0,20.0,542.0,544.0,4.0,93.0,0.0,0.0,279.28,12.0,5.0,2.0,114,Kyler Murray
2,ARI,Drew Anderson,QB,00-0035483,76.0,221.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARI,Alfred Morris,RB,00-0029141,70.0,222.0,7.0,ARI,RB,31.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.40,0.0,0.0,0.0,NaN,NaN
4,ARI,David Johnson,RB,00-0032187,73.0,224.0,4.0,ARI,RB,28.0,13.0,9.0,47.0,36.0,0.0,0.0,0.0,345.0,2.0,94.0,370.0,4.0,141.50,0.0,1.0,1.0,5,David Johnson


In [90]:
print(ff19_rs.shape, ff19_r.shape)

(1035, 68) (1009, 28)


In [91]:
ff20_r = rosters20_rel.merge(ff20, how = 'outer', left_on = 'player_name', \
                             right_on = 'Player', suffixes = ('_20r', '_20')).\
                    reset_index(drop = True).merge(ranks_20[['adp', 'Name']], \
                                            how = 'left', left_on = 'player_name',right_on = 'Name')
ff20_r.drop(columns = 'Player', inplace = True)

ff20_r.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,adp,Name
743,SF,Daniel Helm,TE,00-0035428,76.0,248.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,CLE,Baker Mayfield,QB,00-0034855,73.0,215.0,2.0,CLE,QB,25.0,16.0,16.0,1.0,1.0,3563.0,26.0,486.0,165.0,1.0,54.0,6.0,0.0,248.0,8.0,8.0,4.0,149,Baker Mayfield
465,LV,Devontae Booker,RB,00-0032972,71.0,219.0,4.0,LVR,RB,28.0,16.0,1.0,21.0,17.0,0.0,0.0,0.0,423.0,3.0,93.0,84.0,0.0,69.0,0.0,1.0,0.0,NaN,NaN
197,CLE,Taywan Taylor,WR,00-0033911,71.0,203.0,3.0,CLE,NaN,25.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN
162,CIN,Jacques Patrick,RB,00-0035760,74.0,234.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
ff20_r.shape

(876, 28)

In [93]:
# merging with rosters and adp ranks

ff21_r = rosters21_rel.merge(ff21, how = 'outer', left_on = 'player_name', \
                             right_on = 'Player', suffixes = ('_21r', '_21')) \
                        .reset_index(drop = True).merge(ranks_21[['adp', 'Name']], \
                                            how = 'left', left_on = 'player_name',right_on = 'Name')
ff21_r.drop(columns = 'Player', inplace = True)

ff21_r.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Tgt,Rec,PassingYds,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Int,Fumbles,FumblesLost,adp,Name
5,ARI,Eno Benjamin,RB,00-0036383,69.0,210.0,1.0,ARI,RB,22.0,9.0,0.0,7.0,6.0,0.0,0.0,0.0,118.0,1.0,34.0,42.0,0.0,22.0,0.0,0.0,0.0,NaN,NaN
562,NE,Jonnu Smith,TE,00-0033858,75.0,248.0,4.0,NWE,TE,26.0,16.0,11.0,45.0,28.0,0.0,0.0,0.0,40.0,0.0,9.0,294.0,1.0,39.0,0.0,1.0,0.0,135,Jonnu Smith
301,GB,Equanimeous St. Brown,WR,00-0034279,77.0,214.0,3.0,GNB,WR,25.0,13.0,2.0,17.0,9.0,0.0,0.0,0.0,14.0,0.0,3.0,98.0,0.0,11.0,0.0,0.0,0.0,NaN,NaN
641,NYJ,La'Mical Perine,RB,00-0036269,71.0,220.0,1.0,NYJ,RB,23.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,8.0,0.0,0.0,3.0,0.0,0.0,0.0,NaN,NaN
420,LA,Buddy Howell,RB,00-0034449,73.0,218.0,3.0,LAR,RB,25.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN


In [94]:
ff22_r = rosters22_rel.merge(ff22, how = 'outer', left_on = 'player_name', 
                             right_on = 'Player', suffixes = ('_22r', '_22'))\
            .reset_index(drop = True).merge(ranks_22[['adp', 'Name']], \
                                            how = 'left', left_on = 'player_name',right_on = 'Name')
ff22_r.drop(columns = 'Player', inplace = True)

ff22_r.sample(5)

,team,player_name,position,player_id,height,weight,years_exp,Tm,Pos,Age,G,GS,Cmp,PassingAtt,PassingYds,PassingTD,Int,RushingAtt,RushingYds,Y/A,RushingTD,Tgt,Rec,ReceivingYds,Y/R,ReceivingTD,Fumbles,FumblesLost,TotalTds_22,2PM,2PP,FantasyPoints,PPR,DKPt,FDPt,VBD,PosRank,adp,Name
353,JAX,C.J. Beathard,QB,00-0033936,74.0,215.0,5.0,JAX,QB,29,4,0,7,11,35,0,1,4,-4,-1.00,0,0,0,0,,0,0,0,0,,,-1,-1.0,,,,83,NaN,NaN
575,NO,Tre'Quan Smith,WR,00-0034765,74.0,210.0,4.0,NOR,WR,26,15,6,0,0,0,0,0,0,0,,0,27,19,278,14.63,1,0,0,1,,,34,52.8,55.8,43.3,,113,NaN,NaN
433,LA,Jaquarii Roberson,WR,00-0037574,73.0,185.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465,LV,Austin Walter,RB,00-0035187,68.0,202.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,GB,Samori Toure,WR,00-0037098,73.0,190.0,0.0,GNB,WR,24,11,2,0,0,0,0,0,0,0,,0,10,5,82,16.40,1,1,0,1,,,14,19.2,19.2,16.7,,145,NaN,NaN


In [95]:
# making sure all columns are lower case

ff19_r.columns = map(str.lower, ff19_r.columns)
ff20_r.columns = map(str.lower, ff20_r.columns)
ff21_r.columns = map(str.lower, ff21_r.columns)
ff22_r.columns = map(str.lower, ff22_r.columns)


ff19_rs.columns = map(str.lower, ff19_rs.columns)
ff20_rs.columns = map(str.lower, ff20_rs.columns)
ff21_rs.columns = map(str.lower, ff21_rs.columns)
ff22_rs.columns = map(str.lower, ff22_rs.columns)

In [96]:
ff21_r.head()

,team,player_name,position,player_id,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name
0,ARI,Colt McCoy,QB,00-0027688,73.0,212.0,11.0,ARI,QB,35.0,8.0,3.0,0.0,0.0,740.0,3.0,99.0,37.0,0.0,22.0,0.0,0.0,41.0,1.0,4.0,1.0,NaN,NaN
1,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,2.0,ARI,QB,24.0,14.0,14.0,0.0,0.0,3787.0,24.0,481.0,423.0,5.0,88.0,7.0,0.0,300.0,10.0,13.0,0.0,41,Kyler Murray
2,ARI,Trace McSorley,QB,00-0035146,72.0,202.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARI,James Conner,RB,00-0033553,73.0,233.0,4.0,ARI,RB,26.0,15.0,6.0,39.0,37.0,0.0,0.0,0.0,752.0,15.0,202.0,375.0,3.0,221.0,0.0,2.0,0.0,89,James Conner
4,ARI,Chase Edmonds,RB,00-0034681,69.0,210.0,3.0,ARI,RB,25.0,12.0,11.0,53.0,43.0,0.0,0.0,0.0,592.0,2.0,116.0,311.0,0.0,100.0,0.0,1.0,1.0,60,Chase Edmonds


In [97]:
# drop duplicates from all

ff22_r = ff22_r.drop_duplicates(subset = ['player_name', 'player_id'])
ff21_r = ff21_r.drop_duplicates(subset = ['player_name', 'player_id'])
ff20_r = ff20_r.drop_duplicates(subset = ['player_name', 'player_id'])
ff19_r = ff19_r.drop_duplicates(subset = ['player_name', 'player_id'])

ff22_rs = ff22_rs.drop_duplicates(subset = ['player_name', 'player_id'])
ff21_rs = ff21_rs.drop_duplicates(subset = ['player_name', 'player_id'])
ff20_rs = ff20_rs.drop_duplicates(subset = ['player_name', 'player_id'])
ff19_rs = ff19_rs.drop_duplicates(subset = ['player_name', 'player_id'])


### merging all 4 years together

Now, let's merge these dataframes on Player_id

In [98]:
# concatenating data frames
# adding suffix to 21
# ff21_r = ff21_r.add_suffix('_21')


ff_all = ff19_r.merge(ff20_r, how = 'outer', on = 'player_id', 
                      suffixes = ('_19', '_20')).merge(ff21_r, how = 'outer', on = 'player_id', suffixes = ('', '_21')) \
                .merge(ff22_r, how = 'outer', on = 'player_id', suffixes = ('', '_22'))

ff_all.head()

,team_19,player_name_19,position_19,player_id,height_19,weight_19,years_exp_19,tm_19,pos_19,age_19,g_19,gs_19,tgt_19,rec_19,passingyds_19,passingtd_19,passingatt_19,rushingyds_19,rushingtd_19,rushingatt_19,receivingyds_19,receivingtd_19,fantasypoints_19,int_19,fumbles_19,fumbleslost_19,adp_19,name_19,team_20,player_name_20,position_20,height_20,weight_20,years_exp_20,tm_20,pos_20,age_20,g_20,gs_20,tgt_20,rec_20,passingyds_20,passingtd_20,passingatt_20,rushingyds_20,rushingtd_20,rushingatt_20,receivingyds_20,receivingtd_20,fantasypoints_20,int_20,fumbles_20,fumbleslost_20,adp_20,name_20,team,player_name,position,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,team_22,player_name_22,position_22,height_22,weight_22,years_exp_22,tm_22,pos_22,age_22,g_22,gs_22,cmp,passingatt_22,passingyds_22,passingtd_22,int_22,rushingatt_22,rushingyds_22,y/a,rushingtd_22,tgt_22,rec_22,receivingyds_22,y/r,receivingtd_22,fumbles_22,fumbleslost_22,totaltds_22,2pm,2pp,fantasypoints_22,ppr,dkpt,fdpt,vbd,posrank,adp_22,name_22
0,ARI,Brett Hundley,QB,00-0031589,75.0,226.0,4.0,ARI,QB,26.0,3.0,0.0,0.0,0.0,49.0,0.0,11.0,41.0,0.0,7.0,0.0,0.0,6.06,0.0,0.0,0.0,NaN,NaN,ARI,Brett Hundley,QB,75.0,226.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IND,Brett Hundley,QB,75.0,226.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAL,Brett Hundley,QB,75.0,226.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,0.0,ARI,QB,22.0,16.0,16.0,0.0,0.0,3722.0,20.0,542.0,544.0,4.0,93.0,0.0,0.0,279.28,12.0,5.0,2.0,114,Kyler Murray,ARI,Kyler Murray,QB,70.0,207.0,1.0,ARI,QB,23.0,16.0,16.0,0.0,0.0,3971.0,26.0,558.0,819.0,11.0,133.0,0.0,0.0,379.0,12.0,9.0,4.0,64,Kyler Murray,ARI,Kyler Murray,QB,70.0,207.0,2.0,ARI,QB,24.0,14.0,14.0,0.0,0.0,3787.0,24.0,481.0,423.0,5.0,88.0,7.0,0.0,300.0,10.0,13.0,0.0,41,Kyler Murray,ARI,Kyler Murray,QB,70.0,207.0,3.0,ARI,QB,25,11,11,259,390,2368,14,7,67,418,6.24,3,0,0,0,,0,8,2,3,1,3,201,200.5,215.5,207.5,,18,56,Kyler Murray
2,ARI,Drew Anderson,QB,00-0035483,76.0,221.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARI,Alfred Morris,RB,00-0029141,70.0,222.0,7.0,ARI,RB,31.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.40,0.0,0.0,0.0,NaN,NaN,NYG,Alfred Morris,RB,70.0,224.0,8.0,NYG,RB,32.0,9.0,0.0,4.0,3.0,0.0,0.0,0.0,238.0,1.0,55.0,19.0,1.0,38.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARI,David Johnson,RB,00-0032187,73.0,224.0,4.0,ARI,RB,28.0,13.0,9.0,47.0,36.0,0.0,0.0,0.0,345.0,2.0,94.0,370.0,4.0,141.50,0.0,1.0,1.0,5,David Johnson,HOU,David Johnson,RB,73.0,224.0,5.0,HOU,RB,29.0,12.0,12.0,46.0,33.0,0.0,0.0,0.0,691.0,6.0,147.0,314.0,2.0,147.0,0.0,2.0,1.0,30,David Johnson,HOU,David Johnson,RB,73.0,224.0,6.0,HOU,RB,30.0,13.0,4.0,42.0,32.0,0.0,0.0,0.0,228.0,0.0,67.0,225.0,1.0,49.0,0.0,1.0,1.0,101,David Johnson,NO,David Johnson,RB,73.0,224.0,7.0,NOR,RB,31,5,0,0,0,0,0,0,12,24,2.00,0,4,4,47,11.75,0,1,1,0,,,5,9.1,10.1,7.1,,132,NaN,NaN


In [99]:
ff_all_rs =  ff19_rs.merge(ff20_rs, how = 'outer', on = 'player_id', 
                      suffixes = ('_19', '_20')).merge(ff21_rs, how = 'outer', on = 'player_id', suffixes = ('', '_21')) \
                .merge(ff22_rs, how = 'outer', on = 'player_id', suffixes = ('', '_22'))

ff_all_rs.head()

,team_19,player_name_19,position_19,player_id,height_19,weight_19,years_exp_19,season_19,season_type_19,completions_19,attempts_19,passing_yards_19,passing_tds_19,interceptions_19,sacks_19,sack_yards_19,sack_fumbles_19,sack_fumbles_lost_19,passing_air_yards_19,passing_yards_after_catch_19,passing_first_downs_19,passing_epa_19,passing_2pt_conversions_19,pacr_19,dakota_19,carries_19,rushing_yards_19,rushing_tds_19,rushing_fumbles_19,rushing_fumbles_lost_19,rushing_first_downs_19,rushing_epa_19,rushing_2pt_conversions_19,receptions_19,targets_19,receiving_yards_19,receiving_tds_19,receiving_fumbles_19,receiving_fumbles_lost_19,receiving_air_yards_19,receiving_yards_after_catch_19,receiving_first_downs_19,receiving_epa_19,receiving_2pt_conversions_19,racr_19,target_share_19,air_yards_share_19,wopr_x_19,special_teams_tds_19,fantasy_points_19,fantasy_points_ppr_19,games_19,tgt_sh_19,ay_sh_19,yac_sh_19,wopr_y_19,ry_sh_19,rtd_sh_19,rfd_sh_19,rtdfd_sh_19,dom_19,w8dom_19,yptmpa_19,ppr_sh_19,adp_19,name_19,gs_19,player_19,team_20,player_name_20,position_20,height_20,weight_20,years_exp_20,season_20,season_type_20,completions_20,attempts_20,passing_yards_20,passing_tds_20,interceptions_20,sacks_20,sack_yards_20,sack_fumbles_20,sack_fumbles_lost_20,passing_air_yards_20,passing_yards_after_catch_20,passing_first_downs_20,passing_epa_20,passing_2pt_conversions_20,pacr_20,dakota_20,carries_20,rushing_yards_20,rushing_tds_20,rushing_fumbles_20,rushing_fumbles_lost_20,rushing_first_downs_20,rushing_epa_20,rushing_2pt_conversions_20,receptions_20,targets_20,receiving_yards_20,receiving_tds_20,receiving_fumbles_20,receiving_fumbles_lost_20,receiving_air_yards_20,receiving_yards_after_catch_20,receiving_first_downs_20,receiving_epa_20,receiving_2pt_conversions_20,racr_20,target_share_20,air_yards_share_20,wopr_x_20,special_teams_tds_20,fantasy_points_20,fantasy_points_ppr_20,games_20,tgt_sh_20,ay_sh_20,yac_sh_20,wopr_y_20,ry_sh_20,rtd_sh_20,rfd_sh_20,rtdfd_sh_20,dom_20,w8dom_20,yptmpa_20,ppr_sh_20,adp_20,name_20,gs_20,player_20,team,player_name,position,height,weight,years_exp,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,adp,name,gs,player,team_22,player_name_22,position_22,height_22,weight_22,years_exp_22,season_22,season_type_22,completions_22,attempts_22,passing_yards_22,passing_tds_22,interceptions_22,sacks_22,sack_yards_22,sack_fumbles_22,sack_fumbles_lost_22,passing_air_yards_22,passing_yards_after_catch_22,passing_first_downs_22,passing_epa_22,passing_2pt_conversions_22,pacr_22,dakota_22,carries_22,rushing_yards_22,rushing_tds_22,rushing_fumbles_22,rushing_fumbles_lost_22,rushing_first_downs_22,rushing_epa_22,rushing_2pt_conversions_22,receptions_22,targets_22,receiving_yards_22,receiving_tds_22,receiving_fumbles_22,receiving_fumbles_lost_22,receiving_air_yards_22,receiving_yards_after_catch_22,receiving_first_downs_22,receiving_epa_22,receiving_2pt_conversions_22,racr_22,target_share_22,air_yards_share_22,wopr_x_22,special_teams_tds_22,fantasy_points_22,fantasy_points_ppr_22,games_22,tgt_sh_22,ay_sh_22,yac_sh_22,wopr_y_22,ry_sh_22,rtd_sh_22,rfd_sh_22,rtdfd_sh_22,dom_22,w8dom_22,yptmpa_22,ppr_sh_22,adp_22,name_22,gs_22,player_22
0,ARI,Brett Hundley,QB,00-0031589,75.0,226.0,4.0,2019.0,REG,5.0,11.0,49.0,0.0,0.0,2.0,11.0,0.0,0.0,114.0,34.0,2.0,-3.366053,0.

In [100]:
ff_all.sample(30)

,team_19,player_name_19,position_19,player_id,height_19,weight_19,years_exp_19,tm_19,pos_19,age_19,g_19,gs_19,tgt_19,rec_19,passingyds_19,passingtd_19,passingatt_19,rushingyds_19,rushingtd_19,rushingatt_19,receivingyds_19,receivingtd_19,fantasypoints_19,int_19,fumbles_19,fumbleslost_19,adp_19,name_19,team_20,player_name_20,position_20,height_20,weight_20,years_exp_20,tm_20,pos_20,age_20,g_20,gs_20,tgt_20,rec_20,passingyds_20,passingtd_20,passingatt_20,rushingyds_20,rushingtd_20,rushingatt_20,receivingyds_20,receivingtd_20,fantasypoints_20,int_20,fumbles_20,fumbleslost_20,adp_20,name_20,team,player_name,position,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,team_22,player_name_22,position_22,height_22,weight_22,years_exp_22,tm_22,pos_22,age_22,g_22,gs_22,cmp,passingatt_22,passingyds_22,passingtd_22,int_22,rushingatt_22,rushingyds_22,y/a,rushingtd_22,tgt_22,rec_22,receivingyds_22,y/r,receivingtd_22,fumbles_22,fumbleslost_22,totaltds_22,2pm,2pp,fantasypoints_22,ppr,dkpt,fdpt,vbd,posrank,adp_22,name_22
1270,NaN,NaN,NaN,00-0036721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LA,Warren Jackson,WR,78.0,220.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719,NYG,Scott Simonson,TE,00-0031167,77.0,255.0,5.0,NYG,TE,27.0,5.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,3.10,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,DET,Tommylee Lewis,WR,00-0032881,NaN,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,Tommylee Lewis,WR,67.0,168.0,4.0,NOR,WR,28.0,5.0,0.0,3.0,1.0,0.0,0.0,0.0,14.0,0.0,2.0,5.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,MIA,Tommylee Lewis,WR,67.0,168.0,5.0,MIA,WR,29.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
936,TB,Spencer Schnell,WR,00-0035587,68.0,180.0,0.0,TAM,0,25.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,CHI,Reggie Davis,WR,00-0033815,71.0,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHI,Reggie Davis,WR,71.0,170.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,DEN,Orson Charles,TE,00-0029414,NaN,257.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [101]:
ff_all_rs.sample(30)

,team_19,player_name_19,position_19,player_id,height_19,weight_19,years_exp_19,season_19,season_type_19,completions_19,attempts_19,passing_yards_19,passing_tds_19,interceptions_19,sacks_19,sack_yards_19,sack_fumbles_19,sack_fumbles_lost_19,passing_air_yards_19,passing_yards_after_catch_19,passing_first_downs_19,passing_epa_19,passing_2pt_conversions_19,pacr_19,dakota_19,carries_19,rushing_yards_19,rushing_tds_19,rushing_fumbles_19,rushing_fumbles_lost_19,rushing_first_downs_19,rushing_epa_19,rushing_2pt_conversions_19,receptions_19,targets_19,receiving_yards_19,receiving_tds_19,receiving_fumbles_19,receiving_fumbles_lost_19,receiving_air_yards_19,receiving_yards_after_catch_19,receiving_first_downs_19,receiving_epa_19,receiving_2pt_conversions_19,racr_19,target_share_19,air_yards_share_19,wopr_x_19,special_teams_tds_19,fantasy_points_19,fantasy_points_ppr_19,games_19,tgt_sh_19,ay_sh_19,yac_sh_19,wopr_y_19,ry_sh_19,rtd_sh_19,rfd_sh_19,rtdfd_sh_19,dom_19,w8dom_19,yptmpa_19,ppr_sh_19,adp_19,name_19,gs_19,player_19,team_20,player_name_20,position_20,height_20,weight_20,years_exp_20,season_20,season_type_20,completions_20,attempts_20,passing_yards_20,passing_tds_20,interceptions_20,sacks_20,sack_yards_20,sack_fumbles_20,sack_fumbles_lost_20,passing_air_yards_20,passing_yards_after_catch_20,passing_first_downs_20,passing_epa_20,passing_2pt_conversions_20,pacr_20,dakota_20,carries_20,rushing_yards_20,rushing_tds_20,rushing_fumbles_20,rushing_fumbles_lost_20,rushing_first_downs_20,rushing_epa_20,rushing_2pt_conversions_20,receptions_20,targets_20,receiving_yards_20,receiving_tds_20,receiving_fumbles_20,receiving_fumbles_lost_20,receiving_air_yards_20,receiving_yards_after_catch_20,receiving_first_downs_20,receiving_epa_20,receiving_2pt_conversions_20,racr_20,target_share_20,air_yards_share_20,wopr_x_20,special_teams_tds_20,fantasy_points_20,fantasy_points_ppr_20,games_20,tgt_sh_20,ay_sh_20,yac_sh_20,wopr_y_20,ry_sh_20,rtd_sh_20,rfd_sh_20,rtdfd_sh_20,dom_20,w8dom_20,yptmpa_20,ppr_sh_20,adp_20,name_20,gs_20,player_20,team,player_name,position,height,weight,years_exp,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh,adp,name,gs,player,team_22,player_name_22,position_22,height_22,weight_22,years_exp_22,season_22,season_type_22,completions_22,attempts_22,passing_yards_22,passing_tds_22,interceptions_22,sacks_22,sack_yards_22,sack_fumbles_22,sack_fumbles_lost_22,passing_air_yards_22,passing_yards_after_catch_22,passing_first_downs_22,passing_epa_22,passing_2pt_conversions_22,pacr_22,dakota_22,carries_22,rushing_yards_22,rushing_tds_22,rushing_fumbles_22,rushing_fumbles_lost_22,rushing_first_downs_22,rushing_epa_22,rushing_2pt_conversions_22,receptions_22,targets_22,receiving_yards_22,receiving_tds_22,receiving_fumbles_22,receiving_fumbles_lost_22,receiving_air_yards_22,receiving_yards_after_catch_22,receiving_first_downs_22,receiving_epa_22,receiving_2pt_conversions_22,racr_22,target_share_22,air_yards_share_22,wopr_x_22,special_teams_tds_22,fantasy_points_22,fantasy_points_ppr_22,games_22,tgt_sh_22,ay_sh_22,yac_sh_22,wopr_y_22,ry_sh_22,rtd_sh_22,rfd_sh_22,rtdfd_sh_22,dom_22,w8dom_22,yptmpa_22,ppr_sh_22,adp_22,name_22,gs_22,player_22
960,WAS,Chris Thompson,RB,00-0030404,68.0,195.0,6.0,2019.0,REG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.

## now that we have a 'wide' dataframe as above, let's create a 'long' dataframe

In [102]:
# creating a collective DF as well
# adding the season for each 

ff19_r['season'] = '2019'
ff20_r['season'] = '2020'
ff21_r['season'] = '2021'
ff22_r['season'] = '2022'


ff_all_collective = pd.concat([ff19_r, ff20_r, ff21_r, ff22_r])

ff_all_collective.columns


Index(['team', 'player_name', 'position', 'player_id', 'height', 'weight',
       'years_exp', 'tm', 'pos', 'age', 'g', 'gs', 'tgt', 'rec', 'passingyds',
       'passingtd', 'passingatt', 'rushingyds', 'rushingtd', 'rushingatt',
       'receivingyds', 'receivingtd', 'fantasypoints', 'int', 'fumbles',
       'fumbleslost', 'adp', 'name', 'season', 'cmp', 'y/a', 'y/r',
       'totaltds_22', '2pm', '2pp', 'ppr', 'dkpt', 'fdpt', 'vbd', 'posrank'],
      dtype='object')

After the merges, let's ensure the columns are all relevant and no duplicate information

In [103]:
ff_all.shape

(1501, 120)

In [104]:
ff_all_collective.shape

(3473, 40)

There are still plenty of columns, and plenty to possibly get rid of and clean well before modeling. Before that, we will continue bringing data in

## More to merge

Other areas to consider/add features from :
    - QBR data set (how did current qb ('21) do in 20 and 19?)
    - teams data sets (how well did current team perform offensively last year?)
    - how many returning OL?
    - next gen stats

### QBR Dataset
       - not only for the QB themselves, but for the QB of the team the player will be on in current season
### Team performance
     - how did the team perform that the player is currently on last year?
### Next gen stats
    - how did the player perform by next gen stats?



## QBR Data

### Initial exploration + Cleaning
Getting this ready to merge with the rest of the data

In [105]:
qbr19.head()

,season,season_type,game_week,team_abb,player_id,name_short,rank,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,penalty,qbr_raw,sack,name_first,name_last,name_display,headshot_href,team,qualified
937,2019,Regular,Season Total,BAL,3916387,L. Jackson,1.0,83.0,66.7,613,103.7,55.0,39.1,0,2.2,82.3,-7.4,Lamar,Jackson,Lamar Jackson,https://a.espncdn.com/i/headshots/nfl/players/...,Ravens,True
938,2019,Regular,Season Total,KC,3139477,P. Mahomes,2.0,77.7,55.8,585,97.3,71.6,14.3,0,5.0,79.4,-6.5,Patrick,Mahomes,Patrick Mahomes,https://a.espncdn.com/i/headshots/nfl/players/...,Chiefs,True
939,2019,Regular,Season Total,NO,2580,D. Brees,3.0,73.3,33.7,419,62.6,53.1,1.6,0,1.9,75.7,-6.0,Drew,Brees,Drew Brees,https://a.espncdn.com/i/headshots/nfl/players/...,Saints,True
940,2019,Regular,Season Total,DAL,2577417,D. Prescott,4.0,71.9,48.1,690,93.1,70.7,10.0,0,2.6,72.8,-9.7,Dak,Prescott,Dak Prescott,https://a.espncdn.com/i/headshots/nfl/players/...,Cowboys,True
941,2019,Regular,Season Total,SEA,14881,R. Wilson,5.0,71.5,43.0,674,90.9,58.3,10.6,0,1.5,71.2,-20.6,Russell,Wilson,Russell Wilson,https://a.espncdn.com/i/headshots/nfl/players/...,Seahawks,True


In [106]:
# player_id looks different in qbr than in rosters dataset
# let's see if theres a difference

ff_all[ff_all['player_name_19'] == 'Lamar Jackson']

,team_19,player_name_19,position_19,player_id,height_19,weight_19,years_exp_19,tm_19,pos_19,age_19,g_19,gs_19,tgt_19,rec_19,passingyds_19,passingtd_19,passingatt_19,rushingyds_19,rushingtd_19,rushingatt_19,receivingyds_19,receivingtd_19,fantasypoints_19,int_19,fumbles_19,fumbleslost_19,adp_19,name_19,team_20,player_name_20,position_20,height_20,weight_20,years_exp_20,tm_20,pos_20,age_20,g_20,gs_20,tgt_20,rec_20,passingyds_20,passingtd_20,passingatt_20,rushingyds_20,rushingtd_20,rushingatt_20,receivingyds_20,receivingtd_20,fantasypoints_20,int_20,fumbles_20,fumbleslost_20,adp_20,name_20,team,player_name,position,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,team_22,player_name_22,position_22,height_22,weight_22,years_exp_22,tm_22,pos_22,age_22,g_22,gs_22,cmp,passingatt_22,passingyds_22,passingtd_22,int_22,rushingatt_22,rushingyds_22,y/a,rushingtd_22,tgt_22,rec_22,receivingyds_22,y/r,receivingtd_22,fumbles_22,fumbleslost_22,totaltds_22,2pm,2pp,fantasypoints_22,ppr,dkpt,fdpt,vbd,posrank,adp_22,name_22
73,BAL,Lamar Jackson,QB,00-0034796,74.0,212.0,1.0,BAL,QB,22.0,15.0,15.0,0.0,0.0,3127.0,36.0,401.0,1206.0,7.0,176.0,0.0,0.0,415.68,6.0,9.0,2.0,98,Lamar Jackson,BAL,Lamar Jackson,QB,74.0,212.0,2.0,BAL,QB,23.0,15.0,15.0,0.0,0.0,2757.0,26.0,376.0,1005.0,7.0,159.0,0.0,0.0,333.0,9.0,10.0,4.0,22,Lamar Jackson,BAL,Lamar Jackson,QB,74.0,212.0,3.0,BAL,QB,24.0,12.0,12.0,0.0,0.0,2882.0,16.0,382.0,767.0,2.0,133.0,0.0,0.0,240.0,13.0,6.0,3.0,50,Lamar Jackson,BAL,Lamar Jackson,QB,74.0,212.0,4.0,BAL,QB,25,12,12,203,326,2242,17,7,112,764,6.82,3,0,0,0,,0,5,2,3,,1,236,236.1,251.1,243.1,,14,49,Lamar Jackson


player_id isn't the same between QBR dataset and the rest of our data. Need to join on different keys.

Also, let's see what qbs are "qualified" and see what other columns we can get rid of in 2019 data

In [107]:
qbr19['qualified'].value_counts()

True     42
False    39
Name: qualified, dtype: int64

In [108]:
qbr19['qb_plays'].describe()

count     81.000000
mean     249.000000
std      290.395592
min        1.000000
25%        1.000000
50%       76.000000
75%      588.000000
max      772.000000
Name: qb_plays, dtype: float64

In [109]:
qbr20['qb_plays'].describe()

count     84.000000
mean     246.595238
std      275.832021
min        1.000000
25%        3.000000
50%       81.500000
75%      548.750000
max      763.000000
Name: qb_plays, dtype: float64

In [110]:
# lets get rid of QBs with less than 25 plays

qbr19 = qbr19.loc[qbr19['qb_plays'] >= 25]
qbr20 = qbr20.loc[qbr20['qb_plays'] >= 25]
qbr21 = qbr21.loc[qbr21['qb_plays'] >= 25]
qbr22 = qbr22.loc[qbr22['qb_plays'] >= 25]


# let's get rid of playoffs, as well

qbr19 = qbr19.loc[qbr19['season_type'] == 'Regular']
qbr20 = qbr20.loc[qbr20['season_type'] == 'Regular']
qbr21 = qbr21.loc[qbr21['season_type'] == 'Regular']
qbr22 = qbr22.loc[qbr22['season_type'] == 'Regular']

In [111]:
qbr19['season_type'].value_counts()

Regular    37
Name: season_type, dtype: int64

In [112]:
# let's lessen the columns

qbr19.columns

Index(['season', 'season_type', 'game_week', 'team_abb', 'player_id',
       'name_short', 'rank', 'qbr_total', 'pts_added', 'qb_plays', 'epa_total',
       'pass', 'run', 'exp_sack', 'penalty', 'qbr_raw', 'sack', 'name_first',
       'name_last', 'name_display', 'headshot_href', 'team', 'qualified'],
      dtype='object')

In [113]:
qbr19.describe()

,season,player_id,rank,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,penalty,qbr_raw,sack
count,37.0,3.700000e+01,30.000000,37.000000,37.000000,37.000000,37.00000,37.000000,37.000000,37.0,37.000000,37.000000,37.000000
mean,2019.0,1.584786e+06,15.500000,58.940541,13.437838,518.243243,52.67027,32.602703,5.489189,0.0,2.727027,59.289189,-11.867568
std,0.0,1.681375e+06,8.799491,12.333073,20.645505,220.401327,28.39051,21.011498,8.146976,0.0,1.859159,13.078370,5.787585
min,2019.0,2.330000e+03,1.000000,36.200000,-20.100000,28.000000,3.40000,0.100000,-7.300000,0.0,-0.600000,36.800000,-20.600000
25%,2019.0,1.247100e+04,8.250000,50.800000,0.400000,374.000000,30.30000,13.800000,0.000000,0.0,1.400000,50.200000,-16.700000
50%,2019.0,1.676000e+04,16.000000,57.500000,8.900000,598.000000,56.10000,37.300000,4.500000,0.0,2.600000,58.000000,-12.900000
75%,2019.0,3.115293e+06,22.750000,68.300000,27.000000,674.000000,70.70000,48.700000,9.500000,0.0,4.000000,67.200000,-7.800000
max,2019.0,4.038524e+06,30.000000,84.900000,66.700000,772.000000,103.70000,71.600000,39.100000,0.0,6.400000,85.000000,-0.600000


In [114]:
# dropping columns from DFs

qbr_irrel = ['season_type', 'game_week', 'name_short', 'rank', 'penalty', 'name_first', 'name_last',
                     'headshot_href', 'team', 'qualified']

qbr19.drop(columns = qbr_irrel, inplace = True)
qbr20.drop(columns = qbr_irrel, inplace = True)
qbr21.drop(columns = qbr_irrel, inplace = True)
qbr22.drop(columns = qbr_irrel, inplace = True)

# creating collective of all qbr data
qbr_collective = pd.concat([qbr19, qbr20, qbr21, qbr22])


In [115]:
qbr19.columns

Index(['season', 'team_abb', 'player_id', 'qbr_total', 'pts_added', 'qb_plays',
       'epa_total', 'pass', 'run', 'exp_sack', 'qbr_raw', 'sack',
       'name_display'],
      dtype='object')

In [116]:
# merging these with ff_all

# first merging all qbr dataframes
# player_id is usable for these merges
# outer to ensure we capture rookies or qbs that didn't qualify previous year

qbr_all = qbr19.merge(qbr20, on = 'player_id', how = 'outer', suffixes = ['_19', '_20']) \
                .merge(qbr21, on = 'player_id', how = 'outer', suffixes = ['', '_21']) \
                .merge(qbr22, on = 'player_id' ,how = 'outer', suffixes = ['', '_22'])

In [117]:
qbr_all.columns

Index(['season_19', 'team_abb_19', 'player_id', 'qbr_total_19', 'pts_added_19',
       'qb_plays_19', 'epa_total_19', 'pass_19', 'run_19', 'exp_sack_19',
       'qbr_raw_19', 'sack_19', 'name_display_19', 'season_20', 'team_abb_20',
       'qbr_total_20', 'pts_added_20', 'qb_plays_20', 'epa_total_20',
       'pass_20', 'run_20', 'exp_sack_20', 'qbr_raw_20', 'sack_20',
       'name_display_20', 'season', 'team_abb', 'qbr_total', 'pts_added',
       'qb_plays', 'epa_total', 'pass', 'run', 'exp_sack', 'qbr_raw', 'sack',
       'name_display', 'season_22', 'team_abb_22', 'qbr_total_22',
       'pts_added_22', 'qb_plays_22', 'epa_total_22', 'pass_22', 'run_22',
       'exp_sack_22', 'qbr_raw_22', 'sack_22', 'name_display_22'],
      dtype='object')

In [118]:
len(qbr_all['name_display_19'])

64

In [119]:
# getting list of all QB names from 2019 - 2022

qb_names = []

for i in range(len(qbr_all['name_display_19'])):
    if qbr_all['name_display_19'].isna()[i] == False:
        qb_names.append(qbr_all['name_display_19'][i])
    elif qbr_all['name_display_20'].notnull()[i] == True:
        qb_names.append(qbr_all['name_display_20'][i])
    elif qbr_all['name_display'].notnull()[i] == True:
        qb_names.append(qbr_all['name_display'][i])
    else:
        qb_names.append(qbr_all['name_display_22'][i])

In [120]:
qbr_all['player_name'] = qb_names

In [121]:
qbr_all.head()

,season_19,team_abb_19,player_id,qbr_total_19,pts_added_19,qb_plays_19,epa_total_19,pass_19,run_19,exp_sack_19,qbr_raw_19,sack_19,name_display_19,season_20,team_abb_20,qbr_total_20,pts_added_20,qb_plays_20,epa_total_20,pass_20,run_20,exp_sack_20,qbr_raw_20,sack_20,name_display_20,season,team_abb,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display,season_22,team_abb_22,qbr_total_22,pts_added_22,qb_plays_22,epa_total_22,pass_22,run_22,exp_sack_22,qbr_raw_22,sack_22,name_display_22,player_name
0,2019.0,BAL,3916387,83.0,66.7,613.0,103.7,55.0,39.1,0.0,82.3,-7.4,Lamar Jackson,2020.0,BAL,73.7,44.7,585.0,82.9,50.9,30.8,0.0,74.7,-8.9,Lamar Jackson,2021.0,BAL,50.7,1.3,574.0,62.7,43.7,18.2,0.0,50.8,-13.0,Lamar Jackson,2022.0,BAL,59.4,9.1,478.0,53.0,29.8,22.2,0.0,56.7,-8.9,Lamar Jackson,Lamar Jackson
1,2019.0,KC,3139477,77.7,55.8,585.0,97.3,71.6,14.3,0.0,79.4,-6.5,Patrick Mahomes,2020.0,KC,82.9,86.3,710.0,138.1,116.1,19.1,0.0,84.8,-9.4,Patrick Mahomes,2021.0,KC,62.2,30.1,803.0,108.3,85.5,18.1,0.0,63.0,-9.0,Patrick Mahomes,2022.0,KC,77.6,66.9,763.0,132.8,106.5,24.9,0.0,77.6,-10.7,Patrick Mahomes,Patrick Mahomes
2,2019.0,NO,2580,73.3,33.7,419.0,62.6,53.1,1.6,0.0,75.7,-6.0,Drew Brees,2020.0,NO,74.6,36.4,428.0,66.2,62.5,1.0,0.0,76.9,-6.2,Drew Brees,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drew Brees
3,2019.0,DAL,2577417,71.9,48.1,690.0,93.1,70.7,10.0,0.0,72.8,-9.7,Dak Prescott,2020.0,DAL,78.7,20.8,263.0,36.5,31.2,4.9,0.0,75.4,-4.2,Dak Prescott,2021.0,DAL,54.6,15.2,721.0,85.8,90.2,-7.7,0.0,57.4,-10.7,Dak Prescott,2022.0,DAL,58.3,13.8,480.0,56.6,48.5,5.3,0.0,60.0,-8.3,Dak Prescott,Dak Prescott
4,2019.0,SEA,14881,71.5,43.0,674.0,90.9,58.3,10.6,0.0,71.2,-20.6,Russell Wilson,2020.0,SEA,73.5,48.3,716.0,102.9,88.6,9.1,0.0,72.2,-20.7,Russell Wilson,2021.0,SEA,54.7,3.5,500.0,55.4,45.0,7.8,0.0,52.5,-15.2,Russell Wilson,2022.0,DEN,36.9,-22.3,642.0,37.3,20.0,11.9,0.0,38.0,-24.0,Russell Wilson,Russell Wilson


## Next Gen Data

With NextGen data, we want to have table for each position with all RBs, all QBs, all WRs and all TEs

### Running Backs

In [122]:
ngs_rushing.sample(5)

,season,season_type,week,player_display_name,player_position,team_abbr,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
3368,2021,REG,13,Saquon Barkley,RB,NYG,3.524909,27.272727,2.938727,11,55,49.400874,5.599126,5.000000,0.509011,0.363636,0,00-0034844,Saquon,Barkley,26,S.Barkley
3390,2021,REG,14,Austin Ekeler,RB,LAC,3.194478,16.666667,2.673182,12,67,63.192279,3.807721,5.583333,0.317310,0.500000,1,00-0033699,Austin,Ekeler,30,A.Ekeler
3357,2021,REG,12,A.J. Dillon,RB,GB,3.666957,20.000000,2.570895,20,69,65.929807,3.070193,3.450000,0.153510,0.500000,0,00-0036265,Algiers,Dillon,28,A.Dillon
3753,2022,REG,5,Michael Carter,RB,NYJ,7.267143,50.000000,2.900333,10,21,30.242936,-9.242936,2.100000,-0.924294,0.200000,2,00-0036924,Michael,Carter,32,Mi.Carter
2020,2019,REG,7,LeSean McCoy,RB,KC,3.147188,8.333333,2.913167,12,64,50.867796,13.132204,5.333333,1.094350,0.333333,0,00-0027029,LeSean,McCoy,25,L.McCoy


In [123]:
ngs_rushing.columns

Index(['season', 'season_type', 'week', 'player_display_name',
       'player_position', 'team_abbr', 'efficiency',
       'percent_attempts_gte_eight_defenders', 'avg_time_to_los',
       'rush_attempts', 'rush_yards', 'expected_rush_yards',
       'rush_yards_over_expected', 'avg_rush_yards',
       'rush_yards_over_expected_per_att', 'rush_pct_over_expected',
       'rush_touchdowns', 'player_gsis_id', 'player_first_name',
       'player_last_name', 'player_jersey_number', 'player_short_name'],
      dtype='object')

In [124]:
# we only want regular season

ngs_rushing = ngs_rushing.loc[ngs_rushing['season_type'] == 'REG']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_first_name', 'player_last_name', 'player_jersey_number', 'player_short_name']

ngs_rushing.drop(columns = to_drop, inplace = True)

In [125]:
ngs_rushing['player_position'].unique()

array(['RB', 'HB'], dtype=object)

In [126]:
ngs_rushing['player_position'] = 'RB'

In [127]:
ngs_rb_collective = ngs_rushing.groupby(['season','player_display_name'], as_index = False).mean(numeric_only = True)


In [128]:
ngs_avg_stats_19 = ngs_rb_collective[ngs_rb_collective.season == 2019]
ngs_avg_stats_20 = ngs_rb_collective[ngs_rb_collective.season == 2020]
ngs_avg_stats_21 = ngs_rb_collective[ngs_rb_collective.season == 2021]
ngs_avg_stats_22 = ngs_rb_collective[ngs_rb_collective.season == 2022]

In [129]:
# merging all together to save into one larger DF with columns for each seasons stats

ngs_avg_stats_rb = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_display_name', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_rb = ngs_avg_stats_rb.merge(ngs_avg_stats_21, on = 'player_display_name', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_display_name', how = 'outer', suffixes = ('', '_22'))

In [130]:
# bringing player id back in

ngs_avg_collective_rb = ngs_rb_collective.merge(ngs_rushing[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

ngs_avg_stats_rb = ngs_avg_stats_rb.merge(ngs_rushing[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

In [131]:
ngs_avg_stats_rb

,season_19,player_display_name,efficiency_19,percent_attempts_gte_eight_defenders_19,avg_time_to_los_19,rush_attempts_19,rush_yards_19,expected_rush_yards_19,rush_yards_over_expected_19,avg_rush_yards_19,rush_yards_over_expected_per_att_19,rush_pct_over_expected_19,rush_touchdowns_19,season_20,efficiency_20,percent_attempts_gte_eight_defenders_20,avg_time_to_los_20,rush_attempts_20,rush_yards_20,expected_rush_yards_20,rush_yards_over_expected_20,avg_rush_yards_20,rush_yards_over_expected_per_att_20,rush_pct_over_expected_20,rush_touchdowns_20,season,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,season_22,efficiency_22,percent_attempts_gte_eight_defenders_22,avg_time_to_los_22,rush_attempts_22,rush_yards_22,expected_rush_yards_22,rush_yards_over_expected_22,avg_rush_yards_22,rush_yards_over_expected_per_att_22,rush_pct_over_expected_22,rush_touchdowns_22,player_gsis_id
0,2019.0,Aaron Jones,5.027324,20.654669,2.753791,29.000000,133.625000,121.527039,8.972961,4.380245,0.147533,0.392185,2.000000,2020.0,4.084915,19.359120,2.777621,26.800,147.200000,121.740277,24.126389,5.335387,0.822389,0.416846,1.200000,2021.0,4.079515,14.666474,2.689007,28.272727,130.363636,123.325223,6.674777,4.540392,0.204644,0.359243,0.636364,2022.0,3.992047,7.459673,2.852449,31.666667,172.916667,138.867194,24.882806,5.485285,0.820980,0.497970,0.333333,00-0033293
54,2019.0,Adrian Peterson,4.276318,30.083278,2.871629,29.000000,125.214286,113.238887,11.546828,4.259579,0.347277,0.412202,0.714286,2020.0,4.291046,30.683327,2.648373,32.500,123.625000,139.439282,-18.314282,3.716083,-0.655298,0.257278,1.625000,2021.0,8.602485,38.636364,3.026862,10.500000,18.500000,29.897769,-11.397769,1.777273,-1.085266,0.381818,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0025394
78,2019.0,Alexander Mattison,4.165529,39.126007,2.640365,30.600000,138.000000,114.614364,21.185636,4.386271,0.768133,0.373222,0.400000,2020.0,3.886056,37.250000,2.789159,31.800,147.200000,124.987016,21.012984,4.598929,0.713687,0.426817,0.600000,2021.0,4.978309,27.633302,2.752494,38.333333,144.500000,157.028182,-12.194849,3.622771,-0.518098,0.295075,0.833333,2022.0,2.928519,40.000000,2.457667,10.000000,54.000000,47.921812,6.078188,5.400000,0.607819,0.700000,2.000000,00-0034972
95,2019.0,Alvin Kamara,4.101539,16.564764,2.768313,25.230769,117.307692,113.842147,0.850160,4.694062,0.075997,0.402007,0.615385,2020.0,4.238100,9.622421,2.796077,27.000,134.692308,117.972397,13.642988,4.854057,0.292195,0.418997,2.230769,2021.0,4.893968,34.894280,2.788500,36.307692,137.769231,151.576862,-18.730708,3.666814,-0.728144,0.314561,0.615385,2022.0,4.009596,23.655212,2.766934,34.416667,140.500000,137.807551,2.359115,4.057628,0.072310,0.440785,0.333333,00-0033906
146,2019.0,Austin Ekeler,4.123590,14.343434,3.025735,38.200000,162.200000,171.276102,-18.276102,4.420410,-0.148188,0.383174,1.200000,2020.0,3.980014,10.194888,2.902836,26.875,124.250000,114.388295,9.361705,4.655512,0.508988,0.438242,0.250000,2021.0,3.791187,22.800214,2.680179,26.466667,117.400000,112.861583,3.871750,4.496553,0.163271,0.392382,1.600000,2022.0,4.218072,17.717010,2.739774,27.428571,125.000000,108.337330,14.519813,4.708127,0.639840,0.384139,1.785714,00-0033699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2282,NaN,Rachaad White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0,4.146888,22.889320,2.817926,33.333333,127.666667,129.056464,-1.389797,3.805846,-0.008161,0.352638,0.166667,00-0037256
2288,NaN,Travis Etienne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [132]:
ngs_avg_collective_rb

,season,player_display_name,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id
0,2019,Aaron Jones,5.027324,20.654669,2.753791,29.000000,133.625000,121.527039,8.972961,4.380245,0.147533,0.392185,2.000000,00-0033293
54,2020,Aaron Jones,4.084915,19.359120,2.777621,26.800000,147.200000,121.740277,24.126389,5.335387,0.822389,0.416846,1.200000,00-0033293
108,2021,Aaron Jones,4.079515,14.666474,2.689007,28.272727,130.363636,123.325223,6.674777,4.540392,0.204644,0.359243,0.636364,00-0033293
162,2022,Aaron Jones,3.992047,7.459673,2.852449,31.666667,172.916667,138.867194,24.882806,5.485285,0.820980,0.497970,0.333333,00-0033293
216,2019,Adrian Peterson,4.276318,30.083278,2.871629,29.000000,125.214286,113.238887,11.546828,4.259579,0.347277,0.412202,0.714286,00-0025394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7288,2022,Rachaad White,4.146888,22.889320,2.817926,33.333333,127.666667,129.056464,-1.389797,3.805846,-0.008161,0.352638,0.166667,00-0037256
7294,2022,Travis Etienne,3.773225,27.371651,2.714666,33.416667,168.583333,129.908103,37.175230,5.177648,1.344616,0.480647,0.750000,00-0036973
7306,2022,Tyler Allgeier,4.069969,38.988832,2.860562,28.428571,140.714286,112.790267,21.066876,5.083750,0.773001,0.467975,0.428571,00-0037263
7320,2022,Tyrion Davis-Price,5.813030,42.857143,2.642100,14.000000,33.000000,28.412407,4.587593,2.357143,0.327685,0.428571,0.000000,00-0037827


### Passing Stats

In [133]:
ngs_passing.sample(5)

,season,season_type,week,player_display_name,player_position,team_abbr,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
3075,2021,REG,5,Derek Carr,QB,LV,2.848714,4.036818,8.652857,-4.616039,8.571429,34.599243,-0.975714,35,206,0,1,67.083333,22,62.857143,70.637059,-7.779916,22.523854,50.759142,00-0031280,Derek,Carr,4,D.Carr
1744,2019,REG,0,Case Keenum,QB,WAS,2.753697,5.478375,6.876904,-1.398529,17.479675,50.193737,-1.964100,246,1707,11,4,93.326558,160,65.040650,65.258195,-0.217545,19.376839,50.193737,00-0028986,Casey,Keenum,8,C.Keenum
3214,2021,REG,10,Josh Allen,QB,BUF,2.776000,12.104762,12.859286,-0.754524,25.000000,47.663761,3.787857,28,366,2,1,125.595238,21,75.000000,60.588378,14.411622,26.400926,48.789933,00-0034857,Joshua,Allen,17,J.Allen
1785,2019,REG,1,Drew Brees,QB,NO,2.853220,7.513125,7.487179,0.025946,6.976744,42.794800,-0.846154,43,370,2,1,105.765504,32,74.418605,64.034332,10.384273,21.570813,42.794800,00-0020531,Drew,Brees,9,D.Brees
1888,2019,REG,4,Russell Wilson,QB,SEA,2.938607,6.517727,8.253704,-1.735976,10.714286,36.831531,-1.116667,28,240,1,0,114.285714,22,78.571429,63.788404,14.783025,22.237852,36.831531,00-0029263,Russell,Wilson,3,R.Wilson


In [134]:
# like RBs, we only want regular season

ngs_passing = ngs_passing.loc[ngs_passing['season_type'] == 'REG']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_position', 'player_first_name', 'player_last_name', 
           'player_jersey_number', 'player_short_name']

ngs_passing = ngs_passing.drop(columns = to_drop)

# getting average stats by player by year
ngs_avg_stats_qb_collective = ngs_passing.groupby(['season','player_display_name', 'player_gsis_id'], as_index = False).mean(numeric_only = True)

# breaking into their own tables to then merge side by side with each year
ngs_avg_stats_19 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2019]
ngs_avg_stats_20 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2020]
ngs_avg_stats_21 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2021]
ngs_avg_stats_22 = ngs_avg_stats_qb_collective[ngs_avg_stats_qb_collective.season == 2022]


# now to merge
# larger table with columns for each year's stats
ngs_avg_stats_qb = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_gsis_id', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_qb = ngs_avg_stats_qb.merge(ngs_avg_stats_21, on = 'player_gsis_id', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_gsis_id', how = 'outer', suffixes = ('', '_22'))



In [135]:
# bringing player id back in

ngs_avg_collective_qb = ngs_avg_stats_qb_collective.merge(ngs_passing[['player_display_name', 'player_gsis_id']], 
                                     
                                          on = 'player_display_name', how = 'inner').drop_duplicates()


ngs_avg_stats_qb = ngs_avg_stats_qb.merge(ngs_passing[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

### Wide Receivers

In [136]:
ngs_receiving.sample(5)

,season,season_type,week,player_display_name,player_position,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
4503,2019,REG,0,Sterling Shepard,WR,NYG,5.710500,3.129946,9.937952,17.462354,57,83,68.674699,576.0,3,3.318947,3.177620,0.141328,00-0032385,Sterling,Shepard,87,S.Shepard
9714,2022,REG,9,Jakobi Meyers,WR,NE,3.482000,2.835110,6.218333,27.046031,5,6,83.333333,42.0,0,3.080000,4.107000,-1.027000,00-0034960,Jakobi,Meyers,16,J.Meyers
6301,2020,REG,4,Darren Waller,TE,LV,4.418750,3.219728,4.556364,18.207578,9,11,81.818182,88.0,0,5.974444,6.133041,-0.158597,00-0031610,Darren,Waller,83,D.Waller
7502,2021,REG,0,Diontae Johnson,WR,None,4.636402,3.171973,8.385444,32.444350,107,169,63.313609,1161.0,8,5.141121,5.132355,0.008767,00-0035216,Diontae,Johnson,18,D.Johnson
4919,2019,REG,5,Golden Tate,WR,NYG,5.346667,2.991525,3.460000,5.724844,3,6,50.000000,13.0,0,4.516667,4.138480,0.378187,00-0027891,Golden,Tate,15,G.Tate


In [137]:
# like RBs, we only want regular season

ngs_receiving = ngs_receiving.loc[ngs_receiving['season_type'] == 'REG']

# but, we only want WR for position here, not TE

ngs_wr = ngs_receiving.loc[ngs_receiving['player_position'] == 'WR']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_position', 'player_first_name', 'player_last_name', 
           'player_jersey_number', 'player_short_name']

ngs_wr = ngs_wr.drop(columns = to_drop)

# getting average stats by player by year
ngs_avg_stats_wr_collective = ngs_wr.groupby(['season','player_display_name'], as_index = False).mean(numeric_only = True)

# breaking into their own tables to then merge side by side with each year
ngs_avg_stats_19 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2019]
ngs_avg_stats_20 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2020]
ngs_avg_stats_21 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2021]
ngs_avg_stats_22 = ngs_avg_stats_wr_collective[ngs_avg_stats_wr_collective.season == 2022]


# now to merge
ngs_avg_stats_wr = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_display_name', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_wr = ngs_avg_stats_wr.merge(ngs_avg_stats_21, on = 'player_display_name', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_display_name', how = 'outer', suffixes = ('', '_22'))

In [138]:
# bringing player id back in

ngs_avg_collective_wr = ngs_avg_stats_wr_collective.merge(ngs_receiving[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

ngs_avg_stats_wr = ngs_avg_stats_wr.merge(ngs_wr[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

In [139]:
ngs_avg_stats_wr.head()

,season_19,player_display_name,avg_cushion_19,avg_separation_19,avg_intended_air_yards_19,percent_share_of_intended_air_yards_19,receptions_19,targets_19,catch_percentage_19,yards_19,rec_touchdowns_19,avg_yac_19,avg_expected_yac_19,avg_yac_above_expectation_19,season_20,avg_cushion_20,avg_separation_20,avg_intended_air_yards_20,percent_share_of_intended_air_yards_20,receptions_20,targets_20,catch_percentage_20,yards_20,rec_touchdowns_20,avg_yac_20,avg_expected_yac_20,avg_yac_above_expectation_20,season,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,season_22,avg_cushion_22,avg_separation_22,avg_intended_air_yards_22,percent_share_of_intended_air_yards_22,receptions_22,targets_22,catch_percentage_22,yards_22,rec_touchdowns_22,avg_yac_22,avg_expected_yac_22,avg_yac_above_expectation_22,player_gsis_id
0,2019.0,A.J. Brown,4.998002,2.325755,12.586335,34.786790,9.666667,15.777778,59.668295,194.555556,1.444444,8.392478,4.037331,4.355146,2020.0,5.459333,2.474082,11.095020,36.469486,9.928571,14.857143,66.950284,152.071429,1.571429,6.530935,3.642770,2.888166,2021.0,6.147317,2.680218,12.029758,46.975805,10.416667,17.000000,60.611322,143.500000,0.833333,3.787133,3.663129,0.124003,2022.0,5.641254,2.730035,11.734585,42.014711,10.294118,16.823529,62.001262,175.588235,1.294118,6.811628,4.558055,2.253573,00-0035676
52,2019.0,Adam Humphries,5.925798,2.782947,7.091745,15.480636,11.200000,14.800000,72.411348,113.600000,0.400000,3.276297,3.869869,-0.593572,2020.0,6.292988,2.982571,8.382262,20.966688,5.250000,6.500000,81.547619,50.000000,0.500000,2.221917,2.953779,-0.731862,2021.0,6.306215,3.172473,6.206948,12.752039,9.142857,14.142857,62.015032,85.571429,0.000000,4.168160,4.775217,-0.607057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0032009
68,2019.0,Adam Thielen,6.427153,2.900382,12.714965,41.425522,8.833333,13.833333,63.472222,123.500000,1.666667,4.025802,4.177237,-0.151436,2020.0,5.949418,2.580153,13.131793,38.722475,10.769231,15.769231,66.287352,136.384615,2.076923,3.331271,2.841690,0.489581,2021.0,5.911510,3.163530,9.566764,31.250087,11.636364,16.545455,68.465263,124.727273,1.818182,3.640224,3.466008,0.174215,2022.0,6.134638,2.725963,10.038573,27.393536,9.923077,15.076923,63.417577,100.461538,0.769231,2.359794,3.450323,-1.090528,00-0030035
111,2019.0,Albert Wilson,7.035792,3.490992,4.896085,9.164434,9.875000,13.625000,76.675307,81.750000,0.125000,4.166960,4.095299,0.071661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0,6.722083,3.737623,3.132812,6.062925,4.250000,6.250000,70.000000,37.500000,0.000000,7.047875,6.933007,0.114868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0030669
123,2019.0,Alex Erickson,6.613852,2.550214,9.160670,21.850441,8.444444,15.444444,51.971917,106.555556,0.000000,6.163606,4.362931,1.800674,2020.0,5.635000,3.749726,8.580000,15.986088,6.000000,6.000000,100.000000,88.000000,0.000000,6.086667,6.460935,-0.374268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0032543


### Tight Ends

In [140]:
# now we want TEs

ngs_te = ngs_receiving.loc[ngs_receiving['player_position'] == 'TE']

# jersey number, first last name, and short name don't matter
to_drop = ['week', 'season_type', 'player_position', 'player_first_name', 'player_last_name', 
           'player_jersey_number', 'player_short_name']

ngs_te = ngs_te.drop(columns = to_drop)

# getting average stats by player by year
# saving as collective
ngs_avg_stats_te_collective = ngs_te.groupby(['season','player_display_name'], as_index = False).mean(numeric_only = True)

# breaking into their own tables to then merge side by side with each year
ngs_avg_stats_19 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2019]
ngs_avg_stats_20 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2020]
ngs_avg_stats_21 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2021]
ngs_avg_stats_22 = ngs_avg_stats_te_collective[ngs_avg_stats_te_collective.season == 2022]


# now to merge
ngs_avg_stats_te = ngs_avg_stats_19.merge(ngs_avg_stats_20, on = 'player_display_name', how = 'outer', suffixes = ('_19', '_20'))
ngs_avg_stats_te = ngs_avg_stats_te.merge(ngs_avg_stats_21, on = 'player_display_name', how = 'outer', suffixes = ('', '_21')) \
                    .merge(ngs_avg_stats_22, on = 'player_display_name', how = 'outer', suffixes = ('', '_22'))


In [141]:
# bringing player id back in

ngs_avg_collective_te = ngs_avg_stats_te_collective.merge(ngs_receiving[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

ngs_avg_stats_te = ngs_avg_stats_te.merge(ngs_wr[['player_display_name', 'player_gsis_id']], 
                                                        on = 'player_display_name', how = 'inner').drop_duplicates()

In [142]:
ngs_all_wide = pd.concat([ngs_avg_stats_qb, ngs_avg_stats_rb,
                              ngs_avg_stats_wr, ngs_avg_stats_te])

ngs_all_tall = pd.concat([ngs_avg_collective_qb, ngs_avg_collective_rb,
                         ngs_avg_collective_wr, ngs_avg_collective_te])

ngs_all_tall.head()

,season,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation
0,2019,AJ McCarron,00-0031288,3.023472,4.192857,8.103750,-3.910893,16.666667,34.452502,0.572500,36.000000,225.000000,0.000000,1.000000,65.162037,21.000000,58.333333,60.237448,-1.904115,20.992778,46.981911,00-0031288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Aaron Rodgers,00-0023459,2.895882,5.423481,8.730151,-3.306670,13.803295,41.824807,-0.107682,66.941176,470.823529,3.058824,0.470588,95.671160,41.529412,62.443619,64.125088,-1.681469,23.003235,53.071470,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2020,Aaron Rodgers,00-0023459,2.726101,5.402501,7.986633,-2.584132,10.579508,45.733018,-0.501080,61.882353,505.764706,5.647059,0.588235,119.778930,43.764706,71.132776,66.887073,4.245702,22.344334,53.654674,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,2021,Aaron Rodgers,00-0023459,2.629163,4.861571,7.641316,-2.779745,16.516486,43.783561,-0.641163,62.470588,484.117647,4.352941,0.470588,112.464614,43.058824,69.430188,65.620977,3.809211,21.579879,52.007562,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,2022,Aaron Rodgers,00-0023459,2.701436,5.128770,8.027893,-2.899122,15.095707,40.490173,-0.504206,60.222222,410.555556,2.888889,1.333333,93.531095,38.888889,65.154708,66.243327,-1.088619,21.550354,52.482642,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### QBR, NEXT GEN STATS and TEAM DATA all into 1 DF

In [143]:
pd.set_option('display.max_columns', None)

ff_all.shape

(1501, 120)

In [144]:
# adding in wide all next gen data into ff_all

ff_all = ff_all.merge(ngs_all_wide, left_on = 'player_id', right_on = 'player_gsis_id', how = 'outer')

ff_all.head()

,team_19,player_name_19,position_19,player_id,height_19,weight_19,years_exp_19,tm_19,pos_19,age_19,g_19,gs_19,tgt_19,rec_19,passingyds_19,passingtd_19,passingatt_19,rushingyds_19,rushingtd_19,rushingatt_19,receivingyds_19,receivingtd_19,fantasypoints_19,int_19,fumbles_19,fumbleslost_19,adp_19,name_19,team_20,player_name_20,position_20,height_20,weight_20,years_exp_20,tm_20,pos_20,age_20,g_20,gs_20,tgt_20,rec_20,passingyds_20,passingtd_20,passingatt_20,rushingyds_20,rushingtd_20,rushingatt_20,receivingyds_20,receivingtd_20,fantasypoints_20,int_20,fumbles_20,fumbleslost_20,adp_20,name_20,team,player_name,position,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,team_22,player_name_22,position_22,height_22,weight_22,years_exp_22,tm_22,pos_22,age_22,g_22,gs_22,cmp,passingatt_22,passingyds_22,passingtd_22,int_22,rushingatt_22,rushingyds_22,y/a,rushingtd_22,tgt_22,rec_22,receivingyds_22,y/r,receivingtd_22,fumbles_22,fumbleslost_22,totaltds_22,2pm,2pp,fantasypoints_22,ppr,dkpt,fdpt,vbd,posrank,adp_22,name_22,season_19,player_display_name_19,player_gsis_id_x,avg_time_to_throw_19,avg_completed_air_yards_19,avg_intended_air_yards_19,avg_air_yards_differential_19,aggressiveness_19,max_completed_air_distance_19,avg_air_yards_to_sticks_19,attempts_19,pass_yards_19,pass_touchdowns_19,interceptions_19,passer_rating_19,completions_19,completion_percentage_19,expected_completion_percentage_19,completion_percentage_above_expectation_19,avg_air_distance_19,max_air_distance_19,season_20,player_display_name_20,avg_time_to_throw_20,avg_completed_air_yards_20,avg_intended_air_yards_20,avg_air_yards_differential_20,aggressiveness_20,max_completed_air_distance_20,avg_air_yards_to_sticks_20,attempts_20,pass_yards_20,pass_touchdowns_20,interceptions_20,passer_rating_20,completions_20,completion_percentage_20,expected_completion_percentage_20,completion_percentage_above_expectation_20,avg_air_distance_20,max_air_distance_20,season,player_display_name,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,season_22,player_display_name_22,avg_time_to_throw_22,avg_completed_air_yards_22,avg_intended_air_yards_22,avg_air_yards_differential_22,aggressiveness_22,max_completed_air_distance_22,avg_air_yards_to_sticks_22,attempts_22,pass_yards_22,pass_touchdowns_22,interceptions_22,passer_rating_22,completions_22,completion_percentage_22,expected_completion_percentage_22,completion_percentage_above_expectation_22,avg_air_distance_22,max_air_distance_22,player_gsis_id_y,efficiency_19,percent_attempts_gte_eight_defenders_19,avg_time_to_los_19,rush_attempts_19,rush_yards_19,expected_rush_yards_19,rush_yards_over_expected_19,avg_rush_yards_19,rush_yards_over_expected_per_att_19,rush_pct_over_expected_19,rush_touchdowns_19,efficiency_20,percent_attempts_gte_eight_defenders_20,avg_time_to_los_20,rush_attempts_20,rush_yards_20,expected_rush_yards_20,rush_yards_over_expected_20,avg_rush_yards_20,rush_yards_over_expected_per_att_20,rush_pct_over_expected_20,rush_touchdowns_20,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,efficiency_22,percent_attempts_gte_eight_defenders_22,avg_time_to_los_22,rush_attempts_22,rush_yards_22,expected_rush_yards_22,rush_yards_over_expected_22,avg_rush_yards_22,rush_yards_over_expected_per_att_22,rush_pct_over_expected_22,rush_touchdowns_22,player_gsis_id,avg_cushion_19,avg_separation_19,percent_share_of_intended_air_yards_19,receptions_1

In [145]:
ff_all.shape

(2487, 291)

In [146]:
qbr_all.head()

,season_19,team_abb_19,player_id,qbr_total_19,pts_added_19,qb_plays_19,epa_total_19,pass_19,run_19,exp_sack_19,qbr_raw_19,sack_19,name_display_19,season_20,team_abb_20,qbr_total_20,pts_added_20,qb_plays_20,epa_total_20,pass_20,run_20,exp_sack_20,qbr_raw_20,sack_20,name_display_20,season,team_abb,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display,season_22,team_abb_22,qbr_total_22,pts_added_22,qb_plays_22,epa_total_22,pass_22,run_22,exp_sack_22,qbr_raw_22,sack_22,name_display_22,player_name
0,2019.0,BAL,3916387,83.0,66.7,613.0,103.7,55.0,39.1,0.0,82.3,-7.4,Lamar Jackson,2020.0,BAL,73.7,44.7,585.0,82.9,50.9,30.8,0.0,74.7,-8.9,Lamar Jackson,2021.0,BAL,50.7,1.3,574.0,62.7,43.7,18.2,0.0,50.8,-13.0,Lamar Jackson,2022.0,BAL,59.4,9.1,478.0,53.0,29.8,22.2,0.0,56.7,-8.9,Lamar Jackson,Lamar Jackson
1,2019.0,KC,3139477,77.7,55.8,585.0,97.3,71.6,14.3,0.0,79.4,-6.5,Patrick Mahomes,2020.0,KC,82.9,86.3,710.0,138.1,116.1,19.1,0.0,84.8,-9.4,Patrick Mahomes,2021.0,KC,62.2,30.1,803.0,108.3,85.5,18.1,0.0,63.0,-9.0,Patrick Mahomes,2022.0,KC,77.6,66.9,763.0,132.8,106.5,24.9,0.0,77.6,-10.7,Patrick Mahomes,Patrick Mahomes
2,2019.0,NO,2580,73.3,33.7,419.0,62.6,53.1,1.6,0.0,75.7,-6.0,Drew Brees,2020.0,NO,74.6,36.4,428.0,66.2,62.5,1.0,0.0,76.9,-6.2,Drew Brees,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drew Brees
3,2019.0,DAL,2577417,71.9,48.1,690.0,93.1,70.7,10.0,0.0,72.8,-9.7,Dak Prescott,2020.0,DAL,78.7,20.8,263.0,36.5,31.2,4.9,0.0,75.4,-4.2,Dak Prescott,2021.0,DAL,54.6,15.2,721.0,85.8,90.2,-7.7,0.0,57.4,-10.7,Dak Prescott,2022.0,DAL,58.3,13.8,480.0,56.6,48.5,5.3,0.0,60.0,-8.3,Dak Prescott,Dak Prescott
4,2019.0,SEA,14881,71.5,43.0,674.0,90.9,58.3,10.6,0.0,71.2,-20.6,Russell Wilson,2020.0,SEA,73.5,48.3,716.0,102.9,88.6,9.1,0.0,72.2,-20.7,Russell Wilson,2021.0,SEA,54.7,3.5,500.0,55.4,45.0,7.8,0.0,52.5,-15.2,Russell Wilson,2022.0,DEN,36.9,-22.3,642.0,37.3,20.0,11.9,0.0,38.0,-24.0,Russell Wilson,Russell Wilson


In [147]:
ff_all.sample(25)

,team_19,player_name_19,position_19,player_id,height_19,weight_19,years_exp_19,tm_19,pos_19,age_19,g_19,gs_19,tgt_19,rec_19,passingyds_19,passingtd_19,passingatt_19,rushingyds_19,rushingtd_19,rushingatt_19,receivingyds_19,receivingtd_19,fantasypoints_19,int_19,fumbles_19,fumbleslost_19,adp_19,name_19,team_20,player_name_20,position_20,height_20,weight_20,years_exp_20,tm_20,pos_20,age_20,g_20,gs_20,tgt_20,rec_20,passingyds_20,passingtd_20,passingatt_20,rushingyds_20,rushingtd_20,rushingatt_20,receivingyds_20,receivingtd_20,fantasypoints_20,int_20,fumbles_20,fumbleslost_20,adp_20,name_20,team,player_name,position,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,team_22,player_name_22,position_22,height_22,weight_22,years_exp_22,tm_22,pos_22,age_22,g_22,gs_22,cmp,passingatt_22,passingyds_22,passingtd_22,int_22,rushingatt_22,rushingyds_22,y/a,rushingtd_22,tgt_22,rec_22,receivingyds_22,y/r,receivingtd_22,fumbles_22,fumbleslost_22,totaltds_22,2pm,2pp,fantasypoints_22,ppr,dkpt,fdpt,vbd,posrank,adp_22,name_22,season_19,player_display_name_19,player_gsis_id_x,avg_time_to_throw_19,avg_completed_air_yards_19,avg_intended_air_yards_19,avg_air_yards_differential_19,aggressiveness_19,max_completed_air_distance_19,avg_air_yards_to_sticks_19,attempts_19,pass_yards_19,pass_touchdowns_19,interceptions_19,passer_rating_19,completions_19,completion_percentage_19,expected_completion_percentage_19,completion_percentage_above_expectation_19,avg_air_distance_19,max_air_distance_19,season_20,player_display_name_20,avg_time_to_throw_20,avg_completed_air_yards_20,avg_intended_air_yards_20,avg_air_yards_differential_20,aggressiveness_20,max_completed_air_distance_20,avg_air_yards_to_sticks_20,attempts_20,pass_yards_20,pass_touchdowns_20,interceptions_20,passer_rating_20,completions_20,completion_percentage_20,expected_completion_percentage_20,completion_percentage_above_expectation_20,avg_air_distance_20,max_air_distance_20,season,player_display_name,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,season_22,player_display_name_22,avg_time_to_throw_22,avg_completed_air_yards_22,avg_intended_air_yards_22,avg_air_yards_differential_22,aggressiveness_22,max_completed_air_distance_22,avg_air_yards_to_sticks_22,attempts_22,pass_yards_22,pass_touchdowns_22,interceptions_22,passer_rating_22,completions_22,completion_percentage_22,expected_completion_percentage_22,completion_percentage_above_expectation_22,avg_air_distance_22,max_air_distance_22,player_gsis_id_y,efficiency_19,percent_attempts_gte_eight_defenders_19,avg_time_to_los_19,rush_attempts_19,rush_yards_19,expected_rush_yards_19,rush_yards_over_expected_19,avg_rush_yards_19,rush_yards_over_expected_per_att_19,rush_pct_over_expected_19,rush_touchdowns_19,efficiency_20,percent_attempts_gte_eight_defenders_20,avg_time_to_los_20,rush_attempts_20,rush_yards_20,expected_rush_yards_20,rush_yards_over_expected_20,avg_rush_yards_20,rush_yards_over_expected_per_att_20,rush_pct_over_expected_20,rush_touchdowns_20,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,efficiency_22,percent_attempts_gte_eight_defenders_22,avg_time_to_los_22,rush_attempts_22,rush_yards_22,expected_rush_yards_22,rush_yards_over_expected_22,avg_rush_yards_22,rush_yards_over_expected_per_att_22,rush_pct_over_expected_22,rush_touchdowns_22,player_gsis_id,avg_cushion_19,avg_separation_19,percent_share_of_intended_air_yards_19,receptions_1

In [148]:
# adding in QBR data
# on player name

ff_all = ff_all.merge(qbr_all, left_on = 'player_name_19', right_on = 'player_name', how = 'left')

#### Creating overall collective DF with data from each season, with season as its own columns

DFs we have:
    
    ff_all_collective: all fantasy points and scoring from each player from 2019-2022
    ngs_all_collective: all ngs stats from each player from 2019-2022
    qbr_collective: all qbr stats for each qb from 2019-2022
    team_all_collective: all team stats from 2019-2022
    
We'll need to join these on each other to get stats from each player for each season

In [149]:
# creating overall collective DF with data from each season, with season as its own column

qbr_collective.head()

,season,team_abb,player_id,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display
937,2019,BAL,3916387,83.0,66.7,613,103.7,55.0,39.1,0,82.3,-7.4,Lamar Jackson
938,2019,KC,3139477,77.7,55.8,585,97.3,71.6,14.3,0,79.4,-6.5,Patrick Mahomes
939,2019,NO,2580,73.3,33.7,419,62.6,53.1,1.6,0,75.7,-6.0,Drew Brees
940,2019,DAL,2577417,71.9,48.1,690,93.1,70.7,10.0,0,72.8,-9.7,Dak Prescott
941,2019,SEA,14881,71.5,43.0,674,90.9,58.3,10.6,0,71.2,-20.6,Russell Wilson


In [150]:
# season to int
convert = {'season': int}

team_all_collective = team_all_collective.astype(convert)
ff_all_collective = ff_all_collective.astype(convert)


In [151]:
ff_all_collective.head()

,team,player_name,position,player_id,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,season,cmp,y/a,y/r,totaltds_22,2pm,2pp,ppr,dkpt,fdpt,vbd,posrank
0,ARI,Brett Hundley,QB,00-0031589,75.0,226.0,4.0,ARI,QB,26.0,3.0,0.0,0.0,0.0,49.0,0.0,11.0,41.0,0.0,7.0,0.0,0.0,6.06,0.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,0.0,ARI,QB,22.0,16.0,16.0,0.0,0.0,3722.0,20.0,542.0,544.0,4.0,93.0,0.0,0.0,279.28,12.0,5.0,2.0,114,Kyler Murray,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARI,Drew Anderson,QB,00-0035483,76.0,221.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARI,Alfred Morris,RB,00-0029141,70.0,222.0,7.0,ARI,RB,31.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.4,0.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARI,David Johnson,RB,00-0032187,73.0,224.0,4.0,ARI,RB,28.0,13.0,9.0,47.0,36.0,0.0,0.0,0.0,345.0,2.0,94.0,370.0,4.0,141.5,0.0,1.0,1.0,5,David Johnson,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
ngs_all_tall.head()

,season,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation
0,2019,AJ McCarron,00-0031288,3.023472,4.192857,8.103750,-3.910893,16.666667,34.452502,0.572500,36.000000,225.000000,0.000000,1.000000,65.162037,21.000000,58.333333,60.237448,-1.904115,20.992778,46.981911,00-0031288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Aaron Rodgers,00-0023459,2.895882,5.423481,8.730151,-3.306670,13.803295,41.824807,-0.107682,66.941176,470.823529,3.058824,0.470588,95.671160,41.529412,62.443619,64.125088,-1.681469,23.003235,53.071470,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2020,Aaron Rodgers,00-0023459,2.726101,5.402501,7.986633,-2.584132,10.579508,45.733018,-0.501080,61.882353,505.764706,5.647059,0.588235,119.778930,43.764706,71.132776,66.887073,4.245702,22.344334,53.654674,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,2021,Aaron Rodgers,00-0023459,2.629163,4.861571,7.641316,-2.779745,16.516486,43.783561,-0.641163,62.470588,484.117647,4.352941,0.470588,112.464614,43.058824,69.430188,65.620977,3.809211,21.579879,52.007562,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,2022,Aaron Rodgers,00-0023459,2.701436,5.128770,8.027893,-2.899122,15.095707,40.490173,-0.504206,60.222222,410.555556,2.888889,1.333333,93.531095,38.888889,65.154708,66.243327,-1.088619,21.550354,52.482642,00-0023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
all_collective = ff_all_collective.merge(qbr_collective, left_on = ['player_name', 'season'],
                                        right_on = ['name_display', 'season'], how = 'left') \
                                .merge(ngs_all_tall, left_on = ['player_name', 'season'],
                                      right_on = ['player_display_name', 'season'], how = 'left') \
                                .merge(team_all_collective, left_on = ['team', 'season'], 
                                       right_on = ['Abbreviation', 'season'], how = 'outer',
                                      suffixes = ('', '_tm'))

all_collective.head()

,team,player_name,position,player_id_x,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,season,cmp,y/a,y/r,totaltds_22,2pm,2pp,ppr,dkpt,fdpt,vbd,posrank,team_abb,player_id_y,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,Team,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Abbreviation
0,ARI,Brett Hundley,QB,00-0031589,75.0,226.0,4.0,ARI,QB,26.0,3.0,0.0,0.0,0.0,49.0,0.0,11.0,41.0,0.0,7.0,0.0,0.0,6.06,0.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI
1,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,0.0,ARI,QB,22.0,16.0,16.0,0.0,0.0,3722.0,20.0,542.0,544.0,4.0,93.0,0.0,0.0,279.28,12.0,5.0,2.0,114,Kyler Murray,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARI,3917315.0,57.7,16.2,709.0,71.6,37.3,13.4,0.0,58.0,-17.8,Kyler Murray,Kyler Murray,00-0035228,2.733681,4.914236,6.850385,-1.936149,13.732432,38.64734,-2.111357,63.764706,437.882353,2.352941,1.411765,88.714455,41.058824,65.018637,66.836301,-1.817664,20.623331,45.849949,00-0035228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI
2,ARI,Drew Anderson,QB,00-0035483,76.0,221.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI
3,ARI,Alfred Morris,RB,00-0029141,70.0,222.0,7.0,ARI,RB,31.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.4,0.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI
4,ARI,David Johnson,RB,00-0032187,73.0,224.0,4.0,ARI,RB,28.0,13.0,9.0,47.0,36.0,0.0,0.0,0.0,345.0,2.0,94.0,370.0,4.0,141.5,0.0,1.0,1.0,5,David Johnson,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,David Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [175]:
## Adding new columns to this
## injury data (injuries_by_season)!

all_collective = all_collective.merge(injuries_by_season, how = 'left', left_on = ['player_id_x','season'],
                                        right_on = ['gsis_id', 'season'])
all_collective.sample(10)

,team,player_name,position,player_id_x,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,season,cmp,y/a,y/r,totaltds_22,2pm,2pp,ppr,dkpt,fdpt,vbd,posrank,team_abb,player_id_y,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,Team,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Abbreviation,gsis_id,full_name,games_out,games_questionable,games_out_or_questionable
1307,HOU,Darren Fells,TE,00-0029738,79.0,270.0,7.0,HOU,TE,34.0,16.0,14.0,28.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,312.0,4.0,55.0,0.0,0.0,0.0,NaN,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Darren Fells,NaN,NaN,NaN,7.307143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0029738,6.680000,4.344553,15.302917,6.000000,7.000000,85.714286,85.000000,1.000000,7.926667,7.238801,0.687866,Houston Texans,384.0,326.0,6004.0,4538.0,1466.0,18.0,140.22,-242.84,464.0,6668.0,4104.0,2564.0,9.0,15.43,12.0,0.0,4.0,4.0,HOU,NaN,NaN,NaN,NaN,NaN
583,MIA,De'Lance Turner,RB,00-0034201,71.0,214.0,1.0,MIA,RB,24.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,4.0,0.0,0.0,0.6,0.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miami Dolphins,306.0,315.0,4960.0,3804.0,1156.0,26.0,-28.85,-170.02,494.0,6364.0,4198.0,2166.0,16.0,12.50,11.0,0.0,5.0,5.0,MIA,NaN,NaN,NaN,NaN,NaN
255,DEN,Brandon Allen,QB,00-0032434,74.0,209.0,3.0,DEN,QB,27.0,3.0,3.0,0.0,0.0,515.0,3.0,84.0,39.0,0.0,10.0,0.0,0.0,32.5,2.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brandon Allen,00-0032434,2.540181,6.465931,8.357993,-1.892062,22.192982,40.462934,-0.690754,27.666667,171.666667,1.0,0.666667,73.842836,13.0,48.245614,56.653417,-8.407803,21.932614,48.656271,00-0032434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Denver Broncos,282.0,279.0,4777.0,3115.0,1662.0,16.0,-14.61,-31.98,316.0,5392.0,3609.0,1783.0,17.0,19.21,9.0,0.0,7.0,7.0,DEN,NaN,NaN,NaN,NaN,NaN
3355,PIT,Pat Freiermuth,TE,00-0036894,77.0,258.0,1.0,PIT,TE,24,16,8,98,63,0,0,0,0,0,0,732,2,85,0,0,0,105,Pat Freiermuth,2022,0,,11.62,2,,,148.2,151.2,116.7,,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pat Freiermuth,NaN,NaN,NaN,8.099541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0036894,5.929837,2.670425,22.321201,9.916667,15.333333,63.878968,114.333333,0.333333,5.141362,3.328827,1.812535,Pittsburgh Steelers,77.0,92.0,1479.0,1036.0,443.0,9.0,-25.01,-28.64,128.0,2084.0,1438.0,646.0,10.0,1.03,4.0,0.0,1.0,1.0,PIT,00-0036894,Pat Freiermuth,1.0,0.0,1.0
2739,BAL,Sammy Watkins,WR,00-00313

In [207]:
team_all_collective.head()

,Team,season,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Abbreviation,next_season
0,Arizona Cardinals,2019,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI,2020
1,Arizona Cardinals,2020,410.0,381.0,6153.0,3916.0,2237.0,21.0,103.62,-82.04,367.0,5631.0,3623.0,2008.0,21.0,12.58,8.0,0.0,8.0,8.0,ARI,2021
2,Arizona Cardinals,2021,449.0,367.0,6352.0,4276.0,2076.0,15.0,137.75,-8.05,366.0,5597.0,3645.0,1952.0,27.0,-42.63,6.0,0.0,11.0,11.0,ARI,2022
3,Arizona Cardinals,2022,105.0,112.0,1761.0,1189.0,572.0,3.0,21.28,-34.67,123.0,1728.0,1241.0,487.0,6.0,-4.15,3.0,0.0,2.0,2.0,ARI,2023
4,Atlanta Falcons,2019,381.0,383.0,6075.0,4714.0,1361.0,25.0,95.51,-98.34,399.0,5693.0,3918.0,1775.0,20.0,-22.36,9.0,0.0,7.0,7.0,ATL,2020


In [214]:
# what if i can join the columns i need from here onto all_collective...

team_all_collective['next_season'] = team_all_collective['season'] + 1

team_nxt_season = team_all_collective[['PassY', 'RushY', 'Offense', 'next_season', 'Team']]

In [215]:
team_nxt_season.head()

,PassY,RushY,Offense,next_season,Team
0,3477.0,1990.0,64.36,2020,Arizona Cardinals
1,3916.0,2237.0,103.62,2021,Arizona Cardinals
2,4276.0,2076.0,137.75,2022,Arizona Cardinals
3,1189.0,572.0,21.28,2023,Arizona Cardinals
4,4714.0,1361.0,95.51,2020,Atlanta Falcons


In [219]:
# bringing these in. but it seems 2022 stats are incomplete, again

all_collective = all_collective.merge(team_nxt_season, how = 'left',
                                        left_on = ['Team', 'season'], right_on = ['Team', 'next_season'],
                                       suffixes = ['', '_last_yr_tm_stat'])

In [203]:
# bring in previous year of player's current team's offensive rank as a new feature

# sorting df

test = all_collective.sort_values(['team',  'season'])

test = test.groupby(['team'])
# test['prev_team_output'] = test.groupby(['team', 'season'])['PassY'].shift()

test.head()

,team,player_name,position,player_id_x,height,weight,years_exp,tm,pos,age,g,gs,tgt,rec,passingyds,passingtd,passingatt,rushingyds,rushingtd,rushingatt,receivingyds,receivingtd,fantasypoints,int,fumbles,fumbleslost,adp,name,season,cmp,y/a,y/r,totaltds_22,2pm,2pp,ppr,dkpt,fdpt,vbd,posrank,team_abb,player_id_y,qbr_total,pts_added,qb_plays,epa_total,pass,run,exp_sack,qbr_raw,sack,name_display,player_display_name,player_gsis_id_x,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_differential,aggressiveness,max_completed_air_distance,avg_air_yards_to_sticks,attempts,pass_yards,pass_touchdowns,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,player_gsis_id_y,efficiency,percent_attempts_gte_eight_defenders,avg_time_to_los,rush_attempts,rush_yards,expected_rush_yards,rush_yards_over_expected,avg_rush_yards,rush_yards_over_expected_per_att,rush_pct_over_expected,rush_touchdowns,player_gsis_id,avg_cushion,avg_separation,percent_share_of_intended_air_yards,receptions,targets,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,Team,Pts_scored,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Pts_allowed,Yds_alowed,PassYds_allowed,Rushyds_allowed,Team_takeaways,Sp. Tms,losses,ties,wins,team_wins,Abbreviation,gsis_id,full_name,games_out,games_questionable,games_out_or_questionable
0,ARI,Brett Hundley,QB,00-0031589,75.0,226.0,4.0,ARI,QB,26.0,3.0,0.0,0.0,0.0,49.0,0.0,11.0,41.0,0.0,7.0,0.0,0.0,6.06,0.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI,NaN,NaN,NaN,NaN,NaN
1,ARI,Kyler Murray,QB,00-0035228,70.0,207.0,0.0,ARI,QB,22.0,16.0,16.0,0.0,0.0,3722.0,20.0,542.0,544.0,4.0,93.0,0.0,0.0,279.28,12.0,5.0,2.0,114,Kyler Murray,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARI,3917315.0,57.7,16.2,709.0,71.6,37.3,13.4,0.0,58.0,-17.8,Kyler Murray,Kyler Murray,00-0035228,2.733681,4.914236,6.850385,-1.936149,13.732432,38.647340,-2.111357,63.764706,437.882353,2.352941,1.411765,88.714455,41.058824,65.018637,66.836301,-1.817664,20.623331,45.849949,00-0035228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI,00-0035228,Kyler Murray,0.0,2.0,2.0
2,ARI,Drew Anderson,QB,00-0035483,76.0,221.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI,NaN,NaN,NaN,NaN,NaN
3,ARI,Alfred Morris,RB,00-0029141,70.0,222.0,7.0,ARI,RB,31.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.4,0.0,0.0,0.0,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals,361.0,314.0,5467.0,3477.0,1990.0,18.0,64.36,-173.55,442.0,6432.0,4510.0,1922.0,17.0,19.85,10.0,1.0,5.0,5.5,ARI,NaN,NaN,NaN,NaN,NaN
4,ARI,David Johnson,RB,00-0032187,73.0,224.0,4.0,ARI,RB,28.0,13.0,9.0,47.0,36.0,0.0,0.0,0.0,345.0,2.0,94.0,370.0,4.0,141.5

In [198]:
test.groupby(['team', 'season'])['PassY'].shift()

0          NaN
1       3477.0
2       3477.0
3       3477.0
4       3477.0
         ...  
3474       NaN
2667       NaN
3475       NaN
3472       NaN
3476       NaN
Name: PassY, Length: 3477, dtype: float64

Was going to add rank data, but since 2023 can be changing, it may be better to ingest that in the modeling notebook, or at least, the notebook before it.

In [260]:
all_collective.shape

(3477, 116)

In [261]:
ff_all.shape

(2487, 341)

In [220]:
## Saving the two all encompassing DFs 
# (all, and all_collective -- which are players stats per player over all season, and player stats per season, respectively)
# going to feature engineer and visualize in another notebook

ff_all.to_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/ff_all_position_pts.csv")
all_collective.to_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/ff_all_collective_seasons.csv")

In [221]:
# saving 2023 rank data to utilize in making current year prediction

ranks_23.to_csv("C:/Users/steph/OneDrive/Documents/Data Projects/Fantasy/ff_23_ranks_adp.csv")